In [1]:
from mailbox import _PartialFile
import numpy as np
import xarray as xr
import os
import re
from cdo import *
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind
import skill_metrics as sm
from brokenaxes import brokenaxes

reload(ca)

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo()

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof
import statsmodels.api as sm
import metpy.constants as constants

# 读取观测数据

In [2]:
fpreGPCP = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/GPCP_r144x72_197901-201412.nc"
)
preGPCP = fpreGPCP["precip"]
preGPCP_JJA = ca.p_time(preGPCP, 6, 8, True)
preGPCP_JJA = ca.detrend_dim(preGPCP_JJA, "time", deg=1, demean=False)

fhgtERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/hgt_mon_r144x72_195001-201412.nc")
hgtERA5 = fhgtERA5["z"].sel(time=fhgtERA5["time"].dt.year>=1979)

fuERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/uwind_mon_r144x72_195001-201412.nc"
)
uERA5 = fuERA5["u"].sel(time=fuERA5["time"].dt.year>=1979)

fvERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/vwind_mon_r144x72_195001-201412.nc"
)
vERA5 = fvERA5["v"].sel(time=fvERA5["time"].dt.year>=1979)

fspERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/sp_mon_r144x72_195001-201412.nc"
)
spERA5 = fspERA5["sp"].sel(time=fspERA5["time"].dt.year>=1979)

fqERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/q_mon_r144x72_195001-201412.nc")
qERA5 = fqERA5["q"].sel(time=fqERA5["time"].dt.year>=1979)

fwERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/omega_mon_r144x72_195001-201412.nc")
wERA5 = fwERA5["w"].sel(time=fwERA5["time"].dt.year>=1979)

fsstHad = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/HadISST_r144x72_1870-2020.nc")
sstHad = fsstHad["sst"].sel(time=(fsstHad["time"].dt.year>=1979) & (fsstHad["time"].dt.year<=2014))

hgtERA5_ver_JJA = ca.p_time(hgtERA5, 6, 8, True)
# hgtERA5_ver_JJA = hgtERA5_ver_JJA-hgtERA5_ver_JJA.mean(dim="lon", skipna=True)
hgtERA5_ver_JJA = ca.detrend_dim(hgtERA5_ver_JJA, "time", deg=1, demean=False)

uERA5_ver_MAM, uERA5_ver_JJA, uERA5_ver_SON, uERA5_ver_DJF = ca.mon_to_season(uERA5)
uERA5_ver_JJA_cli = uERA5_ver_JJA.copy()
uERA5_ver_MAM = ca.detrend_dim(uERA5_ver_MAM, "time", deg=1, demean=False)
uERA5_ver_JJA = ca.detrend_dim(uERA5_ver_JJA, "time", deg=1, demean=False)
uERA5_ver_SON = ca.detrend_dim(uERA5_ver_SON, "time", deg=1, demean=False)
uERA5_ver_DJF = ca.detrend_dim(uERA5_ver_DJF, "time", deg=1, demean=False)

vERA5_ver_MAM, vERA5_ver_JJA, vERA5_ver_SON, vERA5_ver_DJF = ca.mon_to_season(vERA5)
vERA5_ver_MAM = ca.detrend_dim(vERA5_ver_MAM, "time", deg=1, demean=False)
vERA5_ver_JJA = ca.detrend_dim(vERA5_ver_JJA, "time", deg=1, demean=False)
vERA5_ver_SON = ca.detrend_dim(vERA5_ver_SON, "time", deg=1, demean=False)
vERA5_ver_DJF = ca.detrend_dim(vERA5_ver_DJF, "time", deg=1, demean=False)

qERA5_ver_JJA = ca.p_time(qERA5, 6, 8, True)
qERA5_ver_JJA = ca.detrend_dim(qERA5_ver_JJA, "time", deg=1, demean=False)

spERA5_MAM, spERA5_JJA, spERA5_SON, spERA5_DJF = ca.mon_to_season(spERA5)
spERA5_MAM = ca.detrend_dim(spERA5_MAM, "time", deg=1, demean=False)
spERA5_JJA = ca.detrend_dim(spERA5_JJA, "time", deg=1, demean=False)
spERA5_SON = ca.detrend_dim(spERA5_SON, "time", deg=1, demean=False)
spERA5_DJF = ca.detrend_dim(spERA5_DJF, "time", deg=1, demean=False)

wERA5_MAM, wERA5_JJA, wERA5_SON, wERA5_DJF = ca.mon_to_season(wERA5)
wERA5_MAM = ca.detrend_dim(wERA5_MAM, "time", deg=1, demean=False)
wERA5_JJA = ca.detrend_dim(wERA5_JJA, "time", deg=1, demean=False)
wERA5_SON = ca.detrend_dim(wERA5_SON, "time", deg=1, demean=False)
wERA5_DJF = ca.detrend_dim(wERA5_DJF, "time", deg=1, demean=False)
sstHad_MAM, sstHad_JJA, sstHad_SON, sstHad_DJF = ca.mon_to_season(sstHad)
sstHad_MAM = ca.detrend_dim(sstHad_MAM, "time", deg=1, demean=False)
sstHad_JJA = ca.detrend_dim(sstHad_JJA, "time", deg=1, demean=False)
sstHad_SON = ca.detrend_dim(sstHad_SON, "time", deg=1, demean=False)
sstHad_DJF = ca.detrend_dim(sstHad_DJF, "time", deg=1, demean=False)
ftERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/temp_mon_r144x72_195001-201412.nc")
tERA5 = ftERA5["t"].sel(time=ftERA5["time"].dt.year>=1979)
tERA5_ver_JJA = ca.p_time(tERA5, 6, 8, True)
tERA5_ver_JJA = ca.detrend_dim(tERA5_ver_JJA, "time", deg=1, demean=False)
divERA5_ver_JJA = ca.cal_divergence(uERA5_ver_JJA.sel(level=[200.0, 850.0]), vERA5_ver_JJA.sel(level=[200.0, 850.0]))
divERA5_ver_JJA = ca.detrend_dim(divERA5_ver_JJA, "time", deg=1, demean=False)

#   calculate the waver vapor vertical intergration
ptop = 1
g = 9.8
ERA5level = qERA5_ver_JJA.coords["level"].where(qERA5_ver_JJA.coords["level"] >= ptop) * 100.0
ERA5level.attrs["units"] = "Pa"
ERA5dp = geocat.comp.dpres_plevel(ERA5level, spERA5_JJA, ptop*100)
ERA5dpg = ERA5dp / g
ERA5dpg.attrs["units"] = "kg/m2"
# calculate the water vapor transport
integration_top = 200
uq_sl_ERA5_JJA = uERA5_ver_JJA.loc[:, integration_top:, :, :] * qERA5_ver_JJA.loc[:, integration_top:, :, :].data * 1000.0
vq_sl_ERA5_JJA = vERA5_ver_JJA.loc[:, integration_top:, :, :] * qERA5_ver_JJA.loc[:, integration_top:, :, :].data * 1000.0
uq_sl_ERA5_JJA.attrs["units"] = "[m/s][g/kg]"
vq_sl_ERA5_JJA.attrs["units"] = "[m/s][g/kg]"
# calculate the whole levels water vapor transport
uqERA5_ver_JJA = (uq_sl_ERA5_JJA * ERA5dpg[:, list(ERA5level).index(integration_top*100):, :, :].data).sum(dim="level") / 1e05
vqERA5_ver_JJA = (vq_sl_ERA5_JJA * ERA5dpg[:, list(ERA5level).index(integration_top*100):, :, :].data).sum(dim="level") / 1e05
uqERA5_ver_JJA.attrs["units"] = "100kg m-1 s-1"
vqERA5_ver_JJA.attrs["units"] = "100kg m-1 s-1"
uqERA5_ver_JJA = ca.detrend_dim(uqERA5_ver_JJA, "time", deg=1, demean=False)
vqERA5_ver_JJA = ca.detrend_dim(vqERA5_ver_JJA, "time", deg=1, demean=False)
# calculate the water vapor transportation divergence
uqvqdivERA5_ver_JJA = ca.cal_divergence(uqERA5_ver_JJA, vqERA5_ver_JJA)
uqvqdivERA5_ver_JJA = ca.detrend_dim(uqvqdivERA5_ver_JJA, "time", deg=1, demean=False)

# 读取数据 CMIP6

## historical

In [3]:
hgthis_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/hgt_historical_r144x72_detrend.nc")
uhis_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/u_historical_r144x72_detrend.nc")
vhis_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/v_historical_r144x72_detrend.nc")
qhis_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/q_historical_r144x72_detrend.nc")
waphis_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/wap_historical_r144x72_detrend.nc")
tahis_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/ta_historical_r144x72_detrend.nc")
prehis_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/pre_historical_r144x72_detrend.nc")
sphis_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/sp_historical_r144x72_detrend.nc")
tshis_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/ts_historical_r144x72_detrend.nc")
toshis_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/tos_historical_r144x72_detrend.nc")
hflshis_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/hfls_historical_r144x72_detrend.nc")

toshis_MAM  = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/tos_historical_r144x72_detrend.nc")
toshis_SON  = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/tos_historical_r144x72_detrend.nc")
toshis_DJF  = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/tos_historical_r144x72_detrend.nc")
waphis_MAM  = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/wap_historical_r144x72_detrend.nc")
waphis_SON  = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/wap_historical_r144x72_detrend.nc")
waphis_DJF  = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/wap_historical_r144x72_detrend.nc")

uq_his_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/uq_historical_r144x72_detrend.nc")
vq_his_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/vq_historical_r144x72_detrend.nc")
uqvqdiv_his_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/uqvqdiv_historical_r144x72_detrend.nc")

## ssp245

In [4]:
hgtssp245_p3_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/zg_ssp245_p3_r144x72_206401-209912.nc")
ussp245_p3_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/ua_ssp245_p3_r144x72_206401-209912.nc")
vssp245_p3_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/va_ssp245_p3_r144x72_206401-209912.nc")
qssp245_p3_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/hus_ssp245_p3_r144x72_206401-209912.nc")

wapssp245_p3_ver_MAM = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/MAM/detrend/wap_ssp245_p3_r144x72_206401-209912.nc")
wapssp245_p3_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/wap_ssp245_p3_r144x72_206401-209912.nc")
wapssp245_p3_ver_SON = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/SON/detrend/wap_ssp245_p3_r144x72_206401-209912.nc")
wapssp245_p3_ver_DJF = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/DJF/detrend/wap_ssp245_p3_r144x72_206401-209912.nc")

tassp245_p3_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/ta_ssp245_p3_r144x72_206401-209912.nc")
pressp245_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/pr_ssp245_p3_r144x72_206401-209912.nc")
spssp245_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/ps_ssp245_p3_r144x72_206401-209912.nc")
tsssp245_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/ts_ssp245_p3_r144x72_206401-209912.nc")

tosssp245_p3_MAM = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/MAM/detrend/tos_ssp245_p3_r144x72_206401-209912.nc")
tosssp245_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/tos_ssp245_p3_r144x72_206401-209912.nc")
tosssp245_p3_SON = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/SON/detrend/tos_ssp245_p3_r144x72_206401-209912.nc")
tosssp245_p3_DJF = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/DJF/detrend/tos_ssp245_p3_r144x72_206401-209912.nc")

hflsssp245_p3_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/hfls_ssp245_p3_r144x72_206401-209912.nc")

uq_ssp245_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/uq_ssp245_p3_r144x72_206401-209912.nc")
vq_ssp245_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/vq_ssp245_p3_r144x72_206401-209912.nc")
uqvqdiv_ssp245_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/uqvqdiv_ssp245_p3_r144x72_206401-209912.nc")

## ssp585

In [5]:
hgtssp585_p3_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/zg_ssp585_p3_r144x72_detrend.nc")
ussp585_p3_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ua_ssp585_p3_r144x72_detrend.nc")
vssp585_p3_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/va_ssp585_p3_r144x72_detrend.nc")
qssp585_p3_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/hus_ssp585_p3_r144x72_detrend.nc")

wapssp585_p3_MAM = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/wap_ssp585_p3_r144x72_detrend.nc")
wapssp585_p3_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/wap_ssp585_p3_r144x72_detrend.nc")
wapssp585_p3_SON = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/wap_ssp585_p3_r144x72_detrend.nc")
wapssp585_p3_DJF = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/wap_ssp585_p3_r144x72_detrend.nc")

tassp585_p3_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ta_ssp585_p3_r144x72_detrend.nc")
pressp585_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/pr_ssp585_p3_r144x72_detrend.nc")
spssp585_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ps_ssp585_p3_r144x72_detrend.nc")
tsssp585_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ts_ssp585_p3_r144x72_detrend.nc")

tosssp585_p3_MAM = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/tos_ssp585_p3_r144x72_detrend.nc")
tosssp585_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/tos_ssp585_p3_r144x72_detrend.nc")
tosssp585_p3_SON = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/tos_ssp585_p3_r144x72_detrend.nc")
tosssp585_p3_DJF = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/tos_ssp585_p3_r144x72_detrend.nc")

hflsssp585_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/hfls_ssp585_p3_r144x72_detrend.nc")

uq_ssp585_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/uq_ssp585_p3_r144x72_206401-209912.nc")
vq_ssp585_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/vq_ssp585_p3_r144x72_206401-209912.nc")
uqvqdiv_ssp585_p3_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/uqvqdiv_ssp585_p3_r144x72_206401-209912.nc")

## 观测偏回归数据

In [6]:
pre_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/pre_north_south_partial_correlation.nc")
hgt_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/hgt_north_south_partial_correlation.nc")
u_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/u_north_south_partial_correlation.nc")
v_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/v_north_south_partial_correlation.nc")
sstDJF_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstDJF_north_south_partial_correlation.nc")
sstMAM_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstMAM_north_south_partial_correlation.nc")
sstJJA_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstJJA_north_south_partial_correlation.nc")
sstSON_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstSON_north_south_partial_correlation.nc")
sstnDJF_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/sstnDJF_north_south_partial_correlation.nc")

uDJF_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uDJF_north_south_partial_correlation.nc")
uMAM_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uMAM_north_south_partial_correlation.nc")
uJJA_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uJJA_north_south_partial_correlation.nc")
uSON_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/uSON_north_south_partial_correlation.nc")
unDJF_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/unDJF_north_south_partial_correlation.nc")

vDJF_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vDJF_north_south_partial_correlation.nc")
vMAM_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vMAM_north_south_partial_correlation.nc")
vJJA_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vJJA_north_south_partial_correlation.nc")
vSON_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vSON_north_south_partial_correlation.nc")
vnDJF_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/vnDJF_north_south_partial_correlation.nc")

spDJF_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spDJF_north_south_partial_correlation.nc")
spMAM_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spMAM_north_south_partial_correlation.nc")
spJJA_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spJJA_north_south_partial_correlation.nc")
spSON_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spSON_north_south_partial_correlation.nc")
spnDJF_north_south_partial_correlation = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/spnDJF_north_south_partial_correlation.nc")

IndR_ERA5_hgt_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_hgt_regress.nc")
IndR_ERA5_u_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_u_regress.nc")
IndR_ERA5_v_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_v_regress.nc")

In [7]:
pre_north_south_slope = pre_north_south_partial_correlation["slope"]
hgt_north_south_slope = hgt_north_south_partial_correlation["slope"]
u_north_south_slope = u_north_south_partial_correlation["slope"]
v_north_south_slope = v_north_south_partial_correlation["slope"]
sstDJF_north_south_slope = sstDJF_north_south_partial_correlation["slope"]
sstMAM_north_south_slope = sstMAM_north_south_partial_correlation["slope"]
sstJJA_north_south_slope = sstJJA_north_south_partial_correlation["slope"]
sstSON_north_south_slope = sstSON_north_south_partial_correlation["slope"]
sstnDJF_north_south_slope = sstnDJF_north_south_partial_correlation["slope"]

spDJF_north_south_slope = spDJF_north_south_partial_correlation["slope"]
spMAM_north_south_slope = spMAM_north_south_partial_correlation["slope"]
spJJA_north_south_slope = spJJA_north_south_partial_correlation["slope"]
spSON_north_south_slope = spSON_north_south_partial_correlation["slope"]
spnDJF_north_south_slope = spnDJF_north_south_partial_correlation["slope"]

uDJF_north_south_slope = uDJF_north_south_partial_correlation["slope"]
uMAM_north_south_slope = uMAM_north_south_partial_correlation["slope"]
uJJA_north_south_slope = uJJA_north_south_partial_correlation["slope"]
uSON_north_south_slope = uSON_north_south_partial_correlation["slope"]
unDJF_north_south_slope = unDJF_north_south_partial_correlation["slope"]

vDJF_north_south_slope = vDJF_north_south_partial_correlation["slope"]
vMAM_north_south_slope = vMAM_north_south_partial_correlation["slope"]
vJJA_north_south_slope = vJJA_north_south_partial_correlation["slope"]
vSON_north_south_slope = vSON_north_south_partial_correlation["slope"]
vnDJF_north_south_slope = vnDJF_north_south_partial_correlation["slope"]

pre_north_south_pvalue = pre_north_south_partial_correlation["pvalue"]
hgt_north_south_pvalue = hgt_north_south_partial_correlation["pvalue"]
u_north_south_pvalue = u_north_south_partial_correlation["pvalue"]
v_north_south_pvalue = v_north_south_partial_correlation["pvalue"]
sstDJF_north_south_pvalue = sstDJF_north_south_partial_correlation["pvalue"]
sstMAM_north_south_pvalue = sstMAM_north_south_partial_correlation["pvalue"]
sstJJA_north_south_pvalue = sstJJA_north_south_partial_correlation["pvalue"]
sstSON_north_south_pvalue = sstSON_north_south_partial_correlation["pvalue"]
sstnDJF_north_south_pvalue = sstnDJF_north_south_partial_correlation["pvalue"]

uDJF_north_south_pvalue = uDJF_north_south_partial_correlation["pvalue"]
uMAM_north_south_pvalue = uMAM_north_south_partial_correlation["pvalue"]
uJJA_north_south_pvalue = uJJA_north_south_partial_correlation["pvalue"]
uSON_north_south_pvalue = uSON_north_south_partial_correlation["pvalue"]
unDJF_north_south_pvalue = unDJF_north_south_partial_correlation["pvalue"]

vDJF_north_south_pvalue = vDJF_north_south_partial_correlation["pvalue"]
vMAM_north_south_pvalue = vMAM_north_south_partial_correlation["pvalue"]
vJJA_north_south_pvalue = vJJA_north_south_partial_correlation["pvalue"]
vSON_north_south_pvalue = vSON_north_south_partial_correlation["pvalue"]
vnDJF_north_south_pvalue = vnDJF_north_south_partial_correlation["pvalue"]

spDJF_north_south_pvalue = spDJF_north_south_partial_correlation["pvalue"]
spMAM_north_south_pvalue = spMAM_north_south_partial_correlation["pvalue"]
spJJA_north_south_pvalue = spJJA_north_south_partial_correlation["pvalue"]
spSON_north_south_pvalue = spSON_north_south_partial_correlation["pvalue"]
spnDJF_north_south_pvalue = spnDJF_north_south_partial_correlation["pvalue"]

IndR_ERA5_hgt_slope = IndR_ERA5_hgt_regress["slope"]
IndR_ERA5_hgt_rvalue = IndR_ERA5_hgt_regress["rvalue"]
IndR_ERA5_hgt_pvalue = IndR_ERA5_hgt_regress["pvalue"]

IndR_ERA5_u_slope = IndR_ERA5_u_regress["slope"]
IndR_ERA5_u_rvalue = IndR_ERA5_u_regress["rvalue"]
IndR_ERA5_u_pvalue = IndR_ERA5_u_regress["pvalue"]

IndR_ERA5_v_slope = IndR_ERA5_v_regress["slope"]
IndR_ERA5_v_rvalue = IndR_ERA5_v_regress["rvalue"]
IndR_ERA5_v_pvalue = IndR_ERA5_v_regress["pvalue"]

# 降水分区

In [8]:
lat = prehis_JJA.coords["lat"]
lon = prehis_JJA.coords["lon"]
# 北部
nIndia_N = 35.0
nIndia_S = 20.0
nIndia_W = 70.0
nIndia_E = 85.0
lat_nIndia_range = lat[(lat >= nIndia_S) & (lat <= nIndia_N)]
lon_nIndia_range = lon[(lon >= nIndia_W) & (lon <= nIndia_E)]

preGPCP_nIndia_JJA = ca.cal_area_weighted_mean(preGPCP_JJA.sel(lat=lat_nIndia_range, lon=lon_nIndia_range))
prehis_nIndia_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_nIndia_range, lon=lon_nIndia_range))
pressp245_p3_nIndia_JJA = ca.cal_area_weighted_mean(pressp245_p3_JJA.sel(lat=lat_nIndia_range, lon=lon_nIndia_range))
pressp585_p3_nIndia_JJA = ca.cal_area_weighted_mean(pressp585_p3_JJA.sel(lat=lat_nIndia_range, lon=lon_nIndia_range))

# detrend
preGPCP_nIndia_JJA_detrend = ca.detrend_dim(preGPCP_nIndia_JJA.copy(), "time", deg=1, demean=True)
prehis_nIndia_JJA_detrend = ca.detrend_dim(prehis_nIndia_JJA.copy(), "time", deg=1, demean=True)
pressp245_p3_nIndia_JJA_detrend = ca.detrend_dim(pressp245_p3_nIndia_JJA.copy(), "time", deg=1, demean=True)
pressp585_p3_nIndia_JJA_detrend = ca.detrend_dim(pressp585_p3_nIndia_JJA.copy(), "time", deg=1, demean=True)

# standardize
preGPCP_nIndia_JJA_nor = ca.standardize(preGPCP_nIndia_JJA_detrend)
prehis_nIndia_JJA_nor = ca.standardize(prehis_nIndia_JJA_detrend)
pressp245_p3_nIndia_JJA_nor = ca.standardize(pressp245_p3_nIndia_JJA_detrend)
pressp585_p3_nIndia_JJA_nor = ca.standardize(pressp585_p3_nIndia_JJA_detrend)

# 南部
sIndia_N = 20.0
sIndia_S = 5.0
sIndia_W = 70.0
sIndia_E = 85.0
lat_sIndia_range = lat[(lat >= sIndia_S) & (lat <= sIndia_N)]
lon_sIndia_range = lon[(lon >= sIndia_W) & (lon <= sIndia_E)]

preGPCP_sIndia_JJA = ca.cal_area_weighted_mean(preGPCP_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))
prehis_sIndia_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))
pressp245_p3_sIndia_JJA = ca.cal_area_weighted_mean(pressp245_p3_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))
pressp585_p3_sIndia_JJA = ca.cal_area_weighted_mean(pressp585_p3_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))

# detrend
preGPCP_sIndia_JJA_detrend = ca.detrend_dim(preGPCP_sIndia_JJA.copy(), "time", deg=1, demean=True)
prehis_sIndia_JJA_detrend = ca.detrend_dim(prehis_sIndia_JJA.copy(), "time", deg=1, demean=True)
pressp245_p3_sIndia_JJA_detrend = ca.detrend_dim(pressp245_p3_sIndia_JJA.copy(), "time", deg=1, demean=True)
pressp585_p3_sIndia_JJA_detrend = ca.detrend_dim(pressp585_p3_sIndia_JJA.copy(), "time", deg=1, demean=True)

# standardize
preGPCP_sIndia_JJA_nor = ca.standardize(preGPCP_sIndia_JJA_detrend)
prehis_sIndia_JJA_nor = ca.standardize(prehis_sIndia_JJA_detrend)
pressp245_p3_sIndia_JJA_nor = ca.standardize(pressp245_p3_sIndia_JJA_detrend)
pressp585_p3_sIndia_JJA_nor = ca.standardize(pressp585_p3_sIndia_JJA_detrend)

# 全印度
India_N = 35.0
India_S = 5.0
India_W = 70.0
India_E = 85.0
lat_India_range = lat[(lat >= India_S) & (lat <= India_N)]
lon_India_range = lon[(lon >= India_W) & (lon <= India_E)]

preGPCP_India_JJA = ca.cal_area_weighted_mean(preGPCP_JJA.sel(lat=lat_India_range, lon=lon_India_range))
prehis_India_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_India_range, lon=lon_India_range))
pressp245_p3_India_JJA = ca.cal_area_weighted_mean(pressp245_p3_JJA.sel(lat=lat_India_range, lon=lon_India_range))
pressp585_p3_India_JJA = ca.cal_area_weighted_mean(pressp585_p3_JJA.sel(lat=lat_India_range, lon=lon_India_range))

# detrend
preGPCP_India_JJA_detrend = ca.detrend_dim(preGPCP_India_JJA.copy(), "time", deg=1, demean=True)
prehis_India_JJA_detrend = ca.detrend_dim(prehis_India_JJA.copy(), "time", deg=1, demean=True)
pressp245_p3_India_JJA_detrend = ca.detrend_dim(pressp245_p3_India_JJA.copy(), "time", deg=1, demean=True)
pressp585_p3_India_JJA_detrend = ca.detrend_dim(pressp585_p3_India_JJA.copy(), "time", deg=1, demean=True)

# standardize
preGPCP_India_JJA_nor = ca.standardize(preGPCP_India_JJA_detrend)
prehis_India_JJA_nor = ca.standardize(prehis_India_JJA_detrend)
pressp245_p3_India_JJA_nor = ca.standardize(pressp245_p3_India_JJA_detrend)
pressp585_p3_India_JJA_nor = ca.standardize(pressp585_p3_India_JJA_detrend)

# # 华北地区
# NC_N = 40.0
# NC_S = 32.5
# NC_W = 105.0
# NC_E = 120.0
# lat_NC_range = lat[(lat >= NC_S) & (lat <= NC_N)]
# lon_NC_range = lon[(lon >= NC_W) & (lon <= NC_E)]
# preGPCP_NC_JJA = ca.cal_area_weighted_mean(preGPCP_JJA.sel(lat=lat_NC_range, lon=lon_NC_range))
# preGPCP_NC_JJA = ca.detrend_dim(preGPCP_NC_JJA, "time", deg=1, demean=True)
# # standardize
# preGPCP_NC_JJA_nor = ca.standardize(preGPCP_NC_JJA)

# prehis_NC_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_NC_range, lon=lon_NC_range))
# prehis_NC_JJA = ca.detrend_dim(prehis_NC_JJA, "time", deg=1, demean=True)
# # standardize
# prehis_NC_JJA_nor = ca.standardize(prehis_NC_JJA)

# # 长江中下游地区
# YZR_N = 32.5
# YZR_S = 27.5
# YZR_W = 105.0
# YZR_E = 120.0
# lat_YZR_range = lat[(lat >= YZR_S) & (lat <= YZR_N)]
# lon_YZR_range = lon[(lon >= YZR_W) & (lon <= YZR_E)]
# preGPCP_YZR_JJA = ca.cal_area_weighted_mean(preGPCP_JJA.sel(lat=lat_YZR_range, lon=lon_YZR_range))
# preGPCP_YZR_JJA = ca.detrend_dim(preGPCP_YZR_JJA, "time", deg=1, demean=True)
# # standardize
# preGPCP_YZR_JJA_nor = ca.standardize(preGPCP_YZR_JJA)

# prehis_YZR_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_YZR_range, lon=lon_YZR_range))
# prehis_YZR_JJA = ca.detrend_dim(prehis_YZR_JJA, "time", deg=1, demean=True)
# # standardize
# prehis_YZR_JJA_nor = ca.standardize(prehis_YZR_JJA)

# # 日本南部
# SJ_N = 35
# SJ_S = 30
# SJ_W = 130.0
# SJ_E = 137.5
# lat_SJ_range = lat[(lat >= SJ_S) & (lat <= SJ_N)]
# lon_SJ_range = lon[(lon >= SJ_W) & (lon <= SJ_E)]
# preGPCP_SJ_JJA = ca.cal_area_weighted_mean(preGPCP_JJA.sel(lat=lat_SJ_range, lon=lon_SJ_range))
# preGPCP_SJ_JJA = ca.detrend_dim(preGPCP_SJ_JJA, "time", deg=1, demean=True)
# # standardize
# preGPCP_SJ_JJA_nor = ca.standardize(preGPCP_SJ_JJA)

# prehis_SJ_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_SJ_range, lon=lon_SJ_range))
# prehis_SJ_JJA = ca.detrend_dim(prehis_SJ_JJA, "time", deg=1, demean=True)
# # standardize
# prehis_SJ_JJA_nor = ca.standardize(prehis_SJ_JJA)


In [9]:
models = hgthis_ver_JJA.models

# 计算降水回归

In [ ]:
(
    nIndR_GPCP_pre_slope,
    nIndR_GPCP_pre_intercept,
    nIndR_GPCP_pre_rvalue,
    nIndR_GPCP_pre_pvalue,
    nIndR_GPCP_pre_hypothesis,
) = ca.dim_linregress(preGPCP_nIndia_JJA_nor, preGPCP_JJA)

(
    sIndR_GPCP_pre_slope,
    sIndR_GPCP_pre_intercept,
    sIndR_GPCP_pre_rvalue,
    sIndR_GPCP_pre_pvalue,
    sIndR_GPCP_pre_hypothesis,
) = ca.dim_linregress(preGPCP_sIndia_JJA_nor, preGPCP_JJA)

(
    IndR_GPCP_pre_slope,
    IndR_GPCP_pre_intercept,
    IndR_GPCP_pre_rvalue,
    IndR_GPCP_pre_pvalue,
    IndR_GPCP_pre_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, preGPCP_JJA)

In [ ]:
(
    nIndR_his_pre_slope,
    nIndR_his_pre_intercept,
    nIndR_his_pre_rvalue,
    nIndR_his_pre_pvalue,
    nIndR_his_pre_hypothesis,
) = ca.dim_linregress(prehis_nIndia_JJA_nor, prehis_JJA)

(
    sIndR_his_pre_slope,
    sIndR_his_pre_intercept,
    sIndR_his_pre_rvalue,
    sIndR_his_pre_pvalue,
    sIndR_his_pre_hypothesis,
) = ca.dim_linregress(prehis_sIndia_JJA_nor, prehis_JJA)

(
    IndR_his_pre_slope,
    IndR_his_pre_intercept,
    IndR_his_pre_rvalue,
    IndR_his_pre_pvalue,
    IndR_his_pre_hypothesis,
) = ca.dim_linregress(prehis_India_JJA_nor, prehis_JJA)

(
    nIndR_ssp245_p3_pre_slope,
    nIndR_ssp245_p3_pre_intercept,
    nIndR_ssp245_p3_pre_rvalue,
    nIndR_ssp245_p3_pre_pvalue,
    nIndR_ssp245_p3_pre_hypothesis,
) = ca.dim_linregress(pressp245_p3_nIndia_JJA_nor, pressp245_p3_JJA)

(
    sIndR_ssp245_p3_pre_slope,
    sIndR_ssp245_p3_pre_intercept,
    sIndR_ssp245_p3_pre_rvalue,
    sIndR_ssp245_p3_pre_pvalue,
    sIndR_ssp245_p3_pre_hypothesis,
) = ca.dim_linregress(pressp245_p3_sIndia_JJA_nor, pressp245_p3_JJA)

(
    IndR_ssp245_p3_pre_slope,
    IndR_ssp245_p3_pre_intercept,
    IndR_ssp245_p3_pre_rvalue,
    IndR_ssp245_p3_pre_pvalue,
    IndR_ssp245_p3_pre_hypothesis,
) = ca.dim_linregress(pressp245_p3_India_JJA_nor, pressp245_p3_JJA)

(
    nIndR_ssp585_p3_pre_slope,
    nIndR_ssp585_p3_pre_intercept,
    nIndR_ssp585_p3_pre_rvalue,
    nIndR_ssp585_p3_pre_pvalue,
    nIndR_ssp585_p3_pre_hypothesis,
) = ca.dim_linregress(pressp585_p3_nIndia_JJA_nor, pressp585_p3_JJA)

(
    sIndR_ssp585_p3_pre_slope,
    sIndR_ssp585_p3_pre_intercept,
    sIndR_ssp585_p3_pre_rvalue,
    sIndR_ssp585_p3_pre_pvalue,
    sIndR_ssp585_p3_pre_hypothesis,
) = ca.dim_linregress(pressp585_p3_sIndia_JJA_nor, pressp585_p3_JJA)

(
    IndR_ssp585_p3_pre_slope,
    IndR_ssp585_p3_pre_intercept,
    IndR_ssp585_p3_pre_rvalue,
    IndR_ssp585_p3_pre_pvalue,
    IndR_ssp585_p3_pre_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, pressp585_p3_JJA)

In [ ]:
nIndR_GPCP_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], nIndR_GPCP_pre_slope.data),
        intercept=(["lat", "lon"], nIndR_GPCP_pre_intercept.data),
        rvalue=(["lat", "lon"], nIndR_GPCP_pre_rvalue.data),
        pvalue=(["lat", "lon"], nIndR_GPCP_pre_pvalue.data),
        hypothesis=(["lat", "lon"], nIndR_GPCP_pre_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of GPCP regress onto 1979-2014"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_GPCP_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_GPCP_pre_regress.nc")
nIndR_GPCP_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_GPCP_pre_regress.nc")
#-------------------------------------------------------
sIndR_GPCP_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], sIndR_GPCP_pre_slope.data),
        intercept=(["lat", "lon"], sIndR_GPCP_pre_intercept.data),
        rvalue=(["lat", "lon"], sIndR_GPCP_pre_rvalue.data),
        pvalue=(["lat", "lon"], sIndR_GPCP_pre_pvalue.data),
        hypothesis=(["lat", "lon"], sIndR_GPCP_pre_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of GPCP regress onto 1979-2014"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_GPCP_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_GPCP_pre_regress.nc")
sIndR_GPCP_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_GPCP_pre_regress.nc")
#-------------------------------------------------------
IndR_GPCP_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndR_GPCP_pre_slope.data),
        intercept=(["lat", "lon"], IndR_GPCP_pre_intercept.data),
        rvalue=(["lat", "lon"], IndR_GPCP_pre_rvalue.data),
        pvalue=(["lat", "lon"], IndR_GPCP_pre_pvalue.data),
        hypothesis=(["lat", "lon"], IndR_GPCP_pre_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of GPCP regress onto 1979-2014"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_GPCP_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_GPCP_pre_regress.nc")
IndR_GPCP_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_GPCP_pre_regress.nc")

In [ ]:
nIndR_his_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_his_pre_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_his_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_his_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_his_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_his_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of his regress onto 1979-2014"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_his_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_his_pre_regress.nc")
nIndR_his_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_his_pre_regress.nc")
#-------------------------------------------------------
sIndR_his_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_his_pre_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_his_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_his_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_his_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_his_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of his regress onto 1979-2014"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_his_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_his_pre_regress.nc")
sIndR_his_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_his_pre_regress.nc")
#-------------------------------------------------------
IndR_his_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_pre_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of his regress onto 1979-2014"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_pre_regress.nc")
IndR_his_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_pre_regress.nc")

In [ ]:
nIndR_ssp245_p3_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_ssp245_p3_pre_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_ssp245_p3_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_ssp245_p3_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_ssp245_p3_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_ssp245_p3_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of ssp245_p3 regress onto 1979-2014"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ssp245_p3_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ssp245_p3_pre_regress.nc")
nIndR_ssp245_p3_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ssp245_p3_pre_regress.nc")
#-------------------------------------------------------
sIndR_ssp245_p3_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_ssp245_p3_pre_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_ssp245_p3_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_ssp245_p3_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_ssp245_p3_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_ssp245_p3_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of ssp245_p3 regress onto 1979-2014"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ssp245_p3_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ssp245_p3_pre_regress.nc")
sIndR_ssp245_p3_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ssp245_p3_pre_regress.nc")
#-------------------------------------------------------
IndR_ssp245_p3_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp245_p3_pre_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp245_p3_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp245_p3_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp245_p3_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp245_p3_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of ssp245_p3 regress onto 1979-2014"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_pre_regress.nc")
IndR_ssp245_p3_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_pre_regress.nc")

In [ ]:
nIndR_ssp585_p3_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_ssp585_p3_pre_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_ssp585_p3_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_ssp585_p3_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of ssp585_p3 regress onto 1979-2014"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ssp585_p3_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ssp585_p3_pre_regress.nc")
nIndR_ssp585_p3_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ssp585_p3_pre_regress.nc")
#-------------------------------------------------------
sIndR_ssp585_p3_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_ssp585_p3_pre_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_ssp585_p3_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_ssp585_p3_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of ssp585_p3 regress onto 1979-2014"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ssp585_p3_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ssp585_p3_pre_regress.nc")
sIndR_ssp585_p3_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ssp585_p3_pre_regress.nc")
#-------------------------------------------------------
IndR_ssp585_p3_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_pre_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of ssp585_p3 regress onto 1979-2014"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_pre_regress.nc")
IndR_ssp585_p3_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_pre_regress.nc")

# 计算其他回归

In [ ]:
preGPCP_nIndia_JJA_nor.coords["time"] = hgtERA5_ver_JJA.time
(
    nIndR_ERA5_hgt_slope,
    nIndR_ERA5_hgt_intercept,
    nIndR_ERA5_hgt_rvalue,
    nIndR_ERA5_hgt_pvalue,
    nIndR_ERA5_hgt_hypothesis,
) = ca.dim_linregress(preGPCP_nIndia_JJA_nor, hgtERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

preGPCP_nIndia_JJA_nor.coords["time"] = uERA5_ver_JJA.time
(
    nIndR_ERA5_uJJA_slope,
    nIndR_ERA5_uJJA_intercept,
    nIndR_ERA5_uJJA_rvalue,
    nIndR_ERA5_uJJA_pvalue,
    nIndR_ERA5_uJJA_hypothesis,
) = ca.dim_linregress(preGPCP_nIndia_JJA_nor, uERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    nIndR_ERA5_vJJA_slope,
    nIndR_ERA5_vJJA_intercept,
    nIndR_ERA5_vJJA_rvalue,
    nIndR_ERA5_vJJA_pvalue,
    nIndR_ERA5_vJJA_hypothesis,
) = ca.dim_linregress(preGPCP_nIndia_JJA_nor, vERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

preGPCP_sIndia_JJA_nor.coords["time"] = hgtERA5_ver_JJA.time
(
    sIndR_ERA5_hgt_slope,
    sIndR_ERA5_hgt_intercept,
    sIndR_ERA5_hgt_rvalue,
    sIndR_ERA5_hgt_pvalue,
    sIndR_ERA5_hgt_hypothesis,
) = ca.dim_linregress(preGPCP_sIndia_JJA_nor, hgtERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

preGPCP_sIndia_JJA_nor.coords["time"] = uERA5_ver_JJA.time
(
    sIndR_ERA5_uJJA_slope,
    sIndR_ERA5_uJJA_intercept,
    sIndR_ERA5_uJJA_rvalue,
    sIndR_ERA5_uJJA_pvalue,
    sIndR_ERA5_uJJA_hypothesis,
) = ca.dim_linregress(preGPCP_sIndia_JJA_nor, uERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    sIndR_ERA5_vJJA_slope,
    sIndR_ERA5_vJJA_intercept,
    sIndR_ERA5_vJJA_rvalue,
    sIndR_ERA5_vJJA_pvalue,
    sIndR_ERA5_vJJA_hypothesis,
) = ca.dim_linregress(preGPCP_sIndia_JJA_nor, vERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    IndR_ERA5_hgt_slope,
    IndR_ERA5_hgt_intercept,
    IndR_ERA5_hgt_rvalue,
    IndR_ERA5_hgt_pvalue,
    IndR_ERA5_hgt_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, hgtERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))
preGPCP_India_JJA_nor.coords["time"] = uERA5_ver_JJA.time
(
    IndR_ERA5_uJJA_slope,
    IndR_ERA5_uJJA_intercept,
    IndR_ERA5_uJJA_rvalue,
    IndR_ERA5_uJJA_pvalue,
    IndR_ERA5_uJJA_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, uERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    IndR_ERA5_vJJA_slope,
    IndR_ERA5_vJJA_intercept,
    IndR_ERA5_vJJA_rvalue,
    IndR_ERA5_vJJA_pvalue,
    IndR_ERA5_vJJA_hypothesis,
) = ca.dim_linregress(preGPCP_India_JJA_nor, vERA5_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

In [ ]:
nIndR_ERA5_uJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], nIndR_ERA5_uJJA_slope.data),
        intercept=(["level", "lat", "lon"], nIndR_ERA5_uJJA_intercept.data),
        rvalue=(["level", "lat", "lon"], nIndR_ERA5_uJJA_rvalue.data),
        pvalue=(["level", "lat", "lon"], nIndR_ERA5_uJJA_pvalue.data),
        hypothesis=(["level", "lat", "lon"], nIndR_ERA5_uJJA_hypothesis.data),
    ),
    coords=dict(
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="w fields of ERA5 regress onto 1979-2014 "),
)
sIndR_ERA5_uJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], sIndR_ERA5_uJJA_slope.data),
        intercept=(["level", "lat", "lon"], sIndR_ERA5_uJJA_intercept.data),
        rvalue=(["level", "lat", "lon"], sIndR_ERA5_uJJA_rvalue.data),
        pvalue=(["level", "lat", "lon"], sIndR_ERA5_uJJA_pvalue.data),
        hypothesis=(["level", "lat", "lon"], sIndR_ERA5_uJJA_hypothesis.data),
    ),
    coords=dict(
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="w fields of ERA5 regress onto 1979-2014 "),
)
IndR_ERA5_uJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], IndR_ERA5_uJJA_slope.data),
        intercept=(["level", "lat", "lon"], IndR_ERA5_uJJA_intercept.data),
        rvalue=(["level", "lat", "lon"], IndR_ERA5_uJJA_rvalue.data),
        pvalue=(["level", "lat", "lon"], IndR_ERA5_uJJA_pvalue.data),
        hypothesis=(["level", "lat", "lon"], IndR_ERA5_uJJA_hypothesis.data),
    ),
    coords=dict(
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="w fields of ERA5 regress onto 1979-2014 "),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ERA5_uJJA_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ERA5_uJJA_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ERA5_uJJA_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ERA5_uJJA_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ERA5_uJJA_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ERA5_uJJA_regress.nc")
  
nIndR_ERA5_uJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ERA5_uJJA_regress.nc")
sIndR_ERA5_uJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ERA5_uJJA_regress.nc")
IndR_ERA5_uJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ERA5_uJJA_regress.nc")

nIndR_ERA5_vJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], nIndR_ERA5_vJJA_slope.data),
        intercept=(["level", "lat", "lon"], nIndR_ERA5_vJJA_intercept.data),
        rvalue=(["level", "lat", "lon"], nIndR_ERA5_vJJA_rvalue.data),
        pvalue=(["level", "lat", "lon"], nIndR_ERA5_vJJA_pvalue.data),
        hypothesis=(["level", "lat", "lon"], nIndR_ERA5_vJJA_hypothesis.data),
    ),
    coords=dict(
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="w fields of ERA5 regress onto 1979-2014 "),
)
sIndR_ERA5_vJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], sIndR_ERA5_vJJA_slope.data),
        intercept=(["level", "lat", "lon"], sIndR_ERA5_vJJA_intercept.data),
        rvalue=(["level", "lat", "lon"], sIndR_ERA5_vJJA_rvalue.data),
        pvalue=(["level", "lat", "lon"], sIndR_ERA5_vJJA_pvalue.data),
        hypothesis=(["level", "lat", "lon"], sIndR_ERA5_vJJA_hypothesis.data),
    ),
    coords=dict(
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="w fields of ERA5 regress onto 1979-2014 "),
)
IndR_ERA5_vJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], IndR_ERA5_vJJA_slope.data),
        intercept=(["level", "lat", "lon"], IndR_ERA5_vJJA_intercept.data),
        rvalue=(["level", "lat", "lon"], IndR_ERA5_vJJA_rvalue.data),
        pvalue=(["level", "lat", "lon"], IndR_ERA5_vJJA_pvalue.data),
        hypothesis=(["level", "lat", "lon"], IndR_ERA5_vJJA_hypothesis.data),
    ),
    coords=dict(
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="w fields of ERA5 regress onto 1979-2014 "),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ERA5_vJJA_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ERA5_vJJA_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ERA5_vJJA_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ERA5_vJJA_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ERA5_vJJA_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ERA5_vJJA_regress.nc")
  
nIndR_ERA5_vJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ERA5_vJJA_regress.nc")

sIndR_ERA5_vJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ERA5_vJJA_regress.nc")

IndR_ERA5_vJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ERA5_vJJA_regress.nc")

level = nIndR_ERA5_hgt_slope.coords["level"]

nIndR_ERA5_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], nIndR_ERA5_hgt_slope.data),
        intercept=(["level", "lat", "lon"], nIndR_ERA5_hgt_intercept.data),
        rvalue=(["level", "lat", "lon"], nIndR_ERA5_hgt_rvalue.data),
        pvalue=(["level", "lat", "lon"], nIndR_ERA5_hgt_pvalue.data),
        hypothesis=(["level", "lat", "lon"], nIndR_ERA5_hgt_hypothesis.data),
    ),
    coords=dict(
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of ERA5 regress onto 1979-2014 "),
)
sIndR_ERA5_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], sIndR_ERA5_hgt_slope.data),
        intercept=(["level", "lat", "lon"], sIndR_ERA5_hgt_intercept.data),
        rvalue=(["level", "lat", "lon"], sIndR_ERA5_hgt_rvalue.data),
        pvalue=(["level", "lat", "lon"], sIndR_ERA5_hgt_pvalue.data),
        hypothesis=(["level", "lat", "lon"], sIndR_ERA5_hgt_hypothesis.data),
    ),
    coords=dict(
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of ERA5 regress onto 1979-2014 "),
)

IndR_ERA5_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["level", "lat", "lon"], IndR_ERA5_hgt_slope.data),
        intercept=(["level", "lat", "lon"], IndR_ERA5_hgt_intercept.data),
        rvalue=(["level", "lat", "lon"], IndR_ERA5_hgt_rvalue.data),
        pvalue=(["level", "lat", "lon"], IndR_ERA5_hgt_pvalue.data),
        hypothesis=(["level", "lat", "lon"], IndR_ERA5_hgt_hypothesis.data),
    ),
    coords=dict(
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of ERA5 regress onto 1979-2014 "),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ERA5_hgt_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ERA5_hgt_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ERA5_hgt_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ERA5_hgt_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ERA5_hgt_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ERA5_hgt_regress.nc")
nIndR_ERA5_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/nIndR_ERA5_hgt_regress.nc")
sIndR_ERA5_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sIndR_ERA5_hgt_regress.nc")
IndR_ERA5_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ERA5_hgt_regress.nc")

## 计算偏回归

In [ ]:
preGPCP_JJA.coords["time"] = hgtERA5_ver_JJA.time
preGPCP_nIndia_JJA_nor.coords["time"] = hgtERA5_ver_JJA.time
preGPCP_sIndia_JJA_nor.coords["time"] = hgtERA5_ver_JJA.time


pre_north_south_avalue, pre_north_south_pvalue = ca.multi_var_regression(preGPCP_JJA, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")
hgt_north_south_avalue, hgt_north_south_pvalue = ca.multi_var_regression(hgtERA5_ver_JJA, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")
divERA5_ver_JJA.coords["time"] = preGPCP_nIndia_JJA_nor.time
div_north_south_avalue, div_north_south_pvalue = ca.multi_var_regression(divERA5_ver_JJA, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")
uqERA5_ver_JJA.coords["time"] = preGPCP_nIndia_JJA_nor.time
vqERA5_ver_JJA.coords["time"] = preGPCP_nIndia_JJA_nor.time
uqvqdivERA5_ver_JJA.coords["time"] = preGPCP_nIndia_JJA_nor.time
uq_north_south_avalue, uq_north_south_pvalue = ca.multi_var_regression(uqERA5_ver_JJA, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")
vq_north_south_avalue, vq_north_south_pvalue = ca.multi_var_regression(vqERA5_ver_JJA, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")
uqvqdiv_north_south_avalue, uqvqdiv_north_south_pvalue = ca.multi_var_regression(uqvqdivERA5_ver_JJA, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")

tmp = sstHad_DJF.copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
sstDJF_north_south_slope, sstDJF_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], preGPCP_sIndia_JJA_nor[1:]], concat_dim="area")

tmp = sstHad_MAM.copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor.time
sstMAM_north_south_slope, sstMAM_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")

tmp = sstHad_JJA.copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor.time
sstJJA_north_south_slope, sstJJA_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")

tmp = sstHad_SON.copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor.time
sstSON_north_south_slope, sstSON_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")

tmp = sstHad_DJF.copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[:-1].time
sstnDJF_north_south_slope, sstnDJF_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[:-1], preGPCP_sIndia_JJA_nor[:-1]], concat_dim="area")

In [ ]:
tmp = uERA5_ver_DJF.sel(level=[200.0, 850.0, 1000.0]).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
uDJF_north_south_slope, uDJF_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], preGPCP_sIndia_JJA_nor[1:]], concat_dim="area")

tmp = uERA5_ver_MAM.sel(level=[200.0, 850.0, 1000.0]).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor.time
uMAM_north_south_slope, uMAM_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")

tmp = uERA5_ver_JJA.sel(level=[200.0, 850.0, 1000.0]).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor.time
uJJA_north_south_slope, uJJA_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")

tmp = uERA5_ver_SON.sel(level=[200.0, 850.0, 1000.0]).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor.time
uSON_north_south_slope, uSON_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")

tmp = uERA5_ver_DJF.sel(level=[200.0, 850.0, 1000.0]).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[:-1].time
unDJF_north_south_slope, unDJF_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[:-1], preGPCP_sIndia_JJA_nor[:-1]], concat_dim="area")

In [ ]:
tmp = vERA5_ver_DJF.sel(level=[200.0, 850.0, 1000.0]).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
vDJF_north_south_slope, vDJF_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], preGPCP_sIndia_JJA_nor[1:]], concat_dim="area")

tmp = vERA5_ver_MAM.sel(level=[200.0, 850.0, 1000.0]).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor.time
vMAM_north_south_slope, vMAM_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")

tmp = vERA5_ver_JJA.sel(level=[200.0, 850.0, 1000.0]).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor.time
vJJA_north_south_slope, vJJA_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")

tmp = vERA5_ver_SON.sel(level=[200.0, 850.0, 1000.0]).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor.time
vSON_north_south_slope, vSON_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")

tmp = vERA5_ver_DJF.sel(level=[200.0, 850.0, 1000.0]).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[:-1].time
vnDJF_north_south_slope, vnDJF_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[:-1], preGPCP_sIndia_JJA_nor[:-1]], concat_dim="area")

In [ ]:
pre_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], pre_north_south_avalue.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], pre_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="precipitation partially regression onto north or south precipitation"),
)

hgt_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], hgt_north_south_avalue.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], hgt_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        level=hgt_north_south_avalue.level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt partially regression onto north or south precipitation"),
)

div_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], div_north_south_avalue.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], div_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        level=div_north_south_avalue.level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="div partially regression onto north or south precipitation"),
)

# u_north_south_partial_correlation = xr.Dataset(
#     data_vars=dict(
#         slope=(["area", "level", "lat", "lon"], u_north_south_avalue.transpose("area", ...).data),
#         pvalue=(["area", "level", "lat", "lon"], u_north_south_pvalue.transpose("area", ...).data),
#     ),
#     coords=dict(
#         area=np.array(["north", "south"]),
#         level=u_north_south_avalue.level.data,
#         lat=lat.data,
#         lon=lon.data,
#     ),
#     attrs=dict(description="u partially regression onto north or south precipitation"),
# )

# v_north_south_partial_correlation = xr.Dataset(
#     data_vars=dict(
#         slope=(["area", "level", "lat", "lon"], v_north_south_avalue.transpose("area", ...).data),
#         pvalue=(["area", "level", "lat", "lon"], v_north_south_pvalue.transpose("area", ...).data),
#     ),
#     coords=dict(
#         area=np.array(["north", "south"]),
#         level=v_north_south_avalue.level.data,
#         lat=lat.data,
#         lon=lon.data,
#     ),
#     attrs=dict(description="v partially regression onto north or south precipitation"),
# )
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/pre_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/pre_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/hgt_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/hgt_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/div_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/div_north_south_partial_correlation.nc")
# if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/u_north_south_partial_correlation.nc"):
#   os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/u_north_south_partial_correlation.nc")
# if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/v_north_south_partial_correlation.nc"):
#   os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/v_north_south_partial_correlation.nc")

pre_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/pre_north_south_partial_correlation.nc")
hgt_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/hgt_north_south_partial_correlation.nc")
div_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/div_north_south_partial_correlation.nc")
# u_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/u_north_south_partial_correlation.nc")
# v_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/v_north_south_partial_correlation.nc")

sstDJF_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], sstDJF_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], sstDJF_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="DJF SST partially regression onto north or south precipitation"),
)

sstMAM_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], sstMAM_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], sstMAM_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="MAM SST partially regression onto north or south precipitation"),
)

sstJJA_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], sstJJA_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], sstJJA_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="JJA SST partially regression onto north or south precipitation"),
)
sstSON_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], sstSON_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], sstSON_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="SON SST partially regression onto north or south precipitation"),
)
sstnDJF_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], sstnDJF_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], sstnDJF_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="nDJF SST partially regression onto north or south precipitation"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstDJF_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstDJF_north_south_partial_correlation.nc")
sstDJF_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstDJF_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstMAM_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstMAM_north_south_partial_correlation.nc")
sstMAM_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstMAM_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstJJA_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstJJA_north_south_partial_correlation.nc")
sstJJA_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstJJA_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstSON_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstSON_north_south_partial_correlation.nc")
sstSON_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstSON_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstnDJF_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstnDJF_north_south_partial_correlation.nc")
sstnDJF_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstnDJF_north_south_partial_correlation.nc")

In [ ]:
uq_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], uq_north_south_avalue.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], uq_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq partially regression onto north or south precipitation"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uq_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uq_north_south_partial_correlation.nc")
uq_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uq_north_south_partial_correlation.nc")

vq_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], vq_north_south_avalue.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], vq_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq partially regression onto north or south precipitation"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vq_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vq_north_south_partial_correlation.nc")
vq_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vq_north_south_partial_correlation.nc")

uqvqdiv_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], uqvqdiv_north_south_avalue.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], uqvqdiv_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv partially regression onto north or south precipitation"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uqvqdiv_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uqvqdiv_north_south_partial_correlation.nc")
uqvqdiv_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uqvqdiv_north_south_partial_correlation.nc")

In [ ]:
uDJF_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], uDJF_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], uDJF_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="DJF u partially regression onto north or south precipitation"),
)

uMAM_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], uMAM_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], uMAM_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="MAM u partially regression onto north or south precipitation"),
)

uJJA_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], uJJA_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], uJJA_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="JJA u partially regression onto north or south precipitation"),
)
uSON_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], uSON_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], uSON_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="SON u partially regression onto north or south precipitation"),
)
unDJF_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], unDJF_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], unDJF_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="nDJF u partially regression onto north or south precipitation"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uDJF_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uDJF_north_south_partial_correlation.nc")
uDJF_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uDJF_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uMAM_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uMAM_north_south_partial_correlation.nc")
uMAM_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uMAM_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uJJA_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uJJA_north_south_partial_correlation.nc")
uJJA_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uJJA_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uSON_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uSON_north_south_partial_correlation.nc")
uSON_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uSON_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/unDJF_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/unDJF_north_south_partial_correlation.nc")
unDJF_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/unDJF_north_south_partial_correlation.nc")

In [ ]:
vDJF_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], vDJF_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], vDJF_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="DJF v partially regression onto north or south precipitation"),
)

vMAM_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], vMAM_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], vMAM_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="MAM v partially regression onto north or south precipitation"),
)

vJJA_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], vJJA_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], vJJA_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="JJA v partially regression onto north or south precipitation"),
)
vSON_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], vSON_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], vSON_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="SON v partially regression onto north or south precipitation"),
)
vnDJF_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "level", "lat", "lon"], vnDJF_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "level", "lat", "lon"], vnDJF_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        level=np.array([200.0, 850.0, 1000.0]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="nDJF v partially regression onto north or south precipitation"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vDJF_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vDJF_north_south_partial_correlation.nc")
vDJF_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vDJF_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vMAM_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vMAM_north_south_partial_correlation.nc")
vMAM_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vMAM_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vJJA_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vJJA_north_south_partial_correlation.nc")
vJJA_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vJJA_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vSON_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vSON_north_south_partial_correlation.nc")
vSON_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vSON_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vnDJF_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vnDJF_north_south_partial_correlation.nc")
vnDJF_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vnDJF_north_south_partial_correlation.nc")

In [ ]:
tmp = wERA5_DJF.sel(level=500.0).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[1:].time
wDJF_north_south_slope, wDJF_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[1:], preGPCP_sIndia_JJA_nor[1:]], concat_dim="area")

tmp = wERA5_MAM.sel(level=500.0).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor.time
wMAM_north_south_slope, wMAM_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")

tmp = wERA5_JJA.sel(level=500.0).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor.time
wJJA_north_south_slope, wJJA_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")

tmp = wERA5_SON.sel(level=500.0).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor.time
wSON_north_south_slope, wSON_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor, preGPCP_sIndia_JJA_nor], concat_dim="area")

tmp = wERA5_DJF.sel(level=500.0).copy()
tmp.coords["time"] = preGPCP_nIndia_JJA_nor[:-1].time
wnDJF_north_south_slope, wnDJF_north_south_pvalue = ca.multi_var_regression(tmp, [preGPCP_nIndia_JJA_nor[:-1], preGPCP_sIndia_JJA_nor[:-1]], concat_dim="area")

In [ ]:
wDJF_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], wDJF_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], wDJF_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="DJF v partially regression onto north or south precipitation"),
)

wMAM_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], wMAM_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], wMAM_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="MAM v partially regression onto north or south precipitation"),
)

wJJA_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], wJJA_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], wJJA_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="JJA v partially regression onto north or south precipitation"),
)
wSON_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], wSON_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], wSON_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="SON v partially regression onto north or south precipitation"),
)
wnDJF_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "lat", "lon"], wnDJF_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "lat", "lon"], wnDJF_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="nDJF v partially regression onto north or south precipitation"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wDJF_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wDJF_north_south_partial_correlation.nc")
wDJF_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wDJF_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wMAM_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wMAM_north_south_partial_correlation.nc")
wMAM_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wMAM_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wJJA_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wJJA_north_south_partial_correlation.nc")
wJJA_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wJJA_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wSON_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wSON_north_south_partial_correlation.nc")
wSON_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wSON_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wnDJF_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wnDJF_north_south_partial_correlation.nc")
wnDJF_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/wnDJF_north_south_partial_correlation.nc")

In [ ]:
# historical
pre_his_north_south_slope, pre_his_north_south_pvalue = ca.multi_var_regression(prehis_JJA, [prehis_nIndia_JJA_nor, prehis_sIndia_JJA_nor], concat_dim="area")
hgt_his_north_south_slope, hgt_his_north_south_pvalue = ca.multi_var_regression(hgthis_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [prehis_nIndia_JJA_nor, prehis_sIndia_JJA_nor], concat_dim="area")
u_his_north_south_slope, u_his_north_south_pvalue = ca.multi_var_regression(uhis_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [prehis_nIndia_JJA_nor, prehis_sIndia_JJA_nor], concat_dim="area")
v_his_north_south_slope, v_his_north_south_pvalue = ca.multi_var_regression(vhis_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [prehis_nIndia_JJA_nor, prehis_sIndia_JJA_nor], concat_dim="area")
uq_his_north_south_slope, uq_his_north_south_pvalue = ca.multi_var_regression(uq_his_JJA, [prehis_nIndia_JJA_nor, prehis_sIndia_JJA_nor], concat_dim="area")
vq_his_north_south_slope, vq_his_north_south_pvalue = ca.multi_var_regression(vq_his_JJA, [prehis_nIndia_JJA_nor, prehis_sIndia_JJA_nor], concat_dim="area")
uqvqdiv_his_north_south_slope, uqvqdiv_his_north_south_pvalue = ca.multi_var_regression(uqvqdiv_his_JJA, [prehis_nIndia_JJA_nor, prehis_sIndia_JJA_nor], concat_dim="area")


# ssp245
pre_ssp245_p3_north_south_slope, pre_ssp245_p3_north_south_pvalue = ca.multi_var_regression(pressp245_p3_JJA, [pressp245_p3_nIndia_JJA_nor, pressp245_p3_sIndia_JJA_nor], concat_dim="area")
hgt_ssp245_p3_north_south_slope, hgt_ssp245_p3_north_south_pvalue = ca.multi_var_regression(hgtssp245_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [pressp245_p3_nIndia_JJA_nor, pressp245_p3_sIndia_JJA_nor], concat_dim="area")
u_ssp245_p3_north_south_slope, u_ssp245_p3_north_south_pvalue = ca.multi_var_regression(ussp245_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [pressp245_p3_nIndia_JJA_nor, pressp245_p3_sIndia_JJA_nor], concat_dim="area")
v_ssp245_p3_north_south_slope, v_ssp245_p3_north_south_pvalue = ca.multi_var_regression(vssp245_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [pressp245_p3_nIndia_JJA_nor, pressp245_p3_sIndia_JJA_nor], concat_dim="area")
uq_ssp245_p3_north_south_slope, uq_ssp245_p3_north_south_pvalue = ca.multi_var_regression(uq_ssp245_p3_JJA, [pressp245_p3_nIndia_JJA_nor, pressp245_p3_sIndia_JJA_nor], concat_dim="area")
vq_ssp245_p3_north_south_slope, vq_ssp245_p3_north_south_pvalue = ca.multi_var_regression(vq_ssp245_p3_JJA, [pressp245_p3_nIndia_JJA_nor, pressp245_p3_sIndia_JJA_nor], concat_dim="area")
uqvqdiv_ssp245_p3_north_south_slope, uqvqdiv_ssp245_p3_north_south_pvalue = ca.multi_var_regression(uqvqdiv_ssp245_p3_JJA, [pressp245_p3_nIndia_JJA_nor, pressp245_p3_sIndia_JJA_nor], concat_dim="area")





# ssp585
pre_ssp585_p3_north_south_slope, pre_ssp585_p3_north_south_pvalue = ca.multi_var_regression(pressp585_p3_JJA, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")
hgt_ssp585_p3_north_south_slope, hgt_ssp585_p3_north_south_pvalue = ca.multi_var_regression(hgtssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")
u_ssp585_p3_north_south_slope, u_ssp585_p3_north_south_pvalue = ca.multi_var_regression(ussp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")
v_ssp585_p3_north_south_slope, v_ssp585_p3_north_south_pvalue = ca.multi_var_regression(vssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")
uq_ssp585_p3_north_south_slope, uq_ssp585_p3_north_south_pvalue = ca.multi_var_regression(uq_ssp585_p3_JJA, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")
vq_ssp585_p3_north_south_slope, vq_ssp585_p3_north_south_pvalue = ca.multi_var_regression(vq_ssp585_p3_JJA, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")
uqvqdiv_ssp585_p3_north_south_slope, uqvqdiv_ssp585_p3_north_south_pvalue = ca.multi_var_regression(uqvqdiv_ssp585_p3_JJA, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")

In [ ]:

# historical
tmp = toshis_DJF.copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
sstDJF_his_north_south_slope, sstDJF_his_north_south_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:]], concat_dim="area")

tmp = toshis_MAM[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
sstMAM_his_north_south_slope, sstMAM_his_north_south_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:]], concat_dim="area")

tmp = toshis_JJA[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
sstJJA_his_north_south_slope, sstJJA_his_north_south_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:]], concat_dim="area")

tmp = toshis_SON[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
sstSON_his_north_south_slope, sstSON_his_north_south_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:]], concat_dim="area")

tmp = toshis_DJF.copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,:-1].time
sstnDJF_his_north_south_slope, sstnDJF_his_north_south_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,:-1], prehis_sIndia_JJA_nor[:,:-1]], concat_dim="area")



In [ ]:

# ssp245
tmp = tosssp245_p3_DJF.copy()
tmp.coords["time"] = pressp245_p3_nIndia_JJA_nor.time
sstDJF_ssp245_p3_north_south_slope, sstDJF_ssp245_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp245_p3_nIndia_JJA_nor, pressp245_p3_sIndia_JJA_nor], concat_dim="area")

tmp = tosssp245_p3_MAM.copy()
tmp.coords["time"] = pressp245_p3_nIndia_JJA_nor.time
sstMAM_ssp245_p3_north_south_slope, sstMAM_ssp245_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp245_p3_nIndia_JJA_nor, pressp245_p3_sIndia_JJA_nor], concat_dim="area")

tmp = tosssp245_p3_JJA.copy()
tmp.coords["time"] = pressp245_p3_nIndia_JJA_nor.time
sstJJA_ssp245_p3_north_south_slope, sstJJA_ssp245_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp245_p3_nIndia_JJA_nor, pressp245_p3_sIndia_JJA_nor], concat_dim="area")

tmp = tosssp245_p3_SON.copy()
tmp.coords["time"] = pressp245_p3_nIndia_JJA_nor.time
sstSON_ssp245_p3_north_south_slope, sstSON_ssp245_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp245_p3_nIndia_JJA_nor, pressp245_p3_sIndia_JJA_nor], concat_dim="area")

tmp = tosssp245_p3_DJF[:,1:].copy()
tmp.coords["time"] = pressp245_p3_nIndia_JJA_nor[:,:-1].time
sstnDJF_ssp245_p3_north_south_slope, sstnDJF_ssp245_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp245_p3_nIndia_JJA_nor[:,:-1], pressp245_p3_sIndia_JJA_nor[:,:-1]], concat_dim="area")

In [ ]:

# ssp585
tmp = tosssp585_p3_DJF.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstDJF_ssp585_p3_north_south_slope, sstDJF_ssp585_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")

tmp = tosssp585_p3_MAM.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstMAM_ssp585_p3_north_south_slope, sstMAM_ssp585_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")

tmp = tosssp585_p3_JJA.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstJJA_ssp585_p3_north_south_slope, sstJJA_ssp585_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")

tmp = tosssp585_p3_SON.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstSON_ssp585_p3_north_south_slope, sstSON_ssp585_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")

tmp = tosssp585_p3_DJF[:,1:].copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor[:,:-1].time
sstnDJF_ssp585_p3_north_south_slope, sstnDJF_ssp585_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor[:,:-1], pressp585_p3_sIndia_JJA_nor[:,:-1]], concat_dim="area")

In [ ]:
uq_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], uq_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], uq_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=uq_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq partially regression onto north or south uq"),
)
vq_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], vq_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], vq_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=vq_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq partially regression onto north or south vq"),
)
uqvqdiv_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], uqvqdiv_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], uqvqdiv_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=uqvqdiv_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv partially regression onto north or south uqvqdiv"),
)

uq_ssp245_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], uq_ssp245_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], uq_ssp245_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=uq_ssp245_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq partially regression onto north or south uq"),
)
vq_ssp245_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], vq_ssp245_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], vq_ssp245_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=vq_ssp245_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq partially regression onto north or south vq"),
)
uqvqdiv_ssp245_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], uqvqdiv_ssp245_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], uqvqdiv_ssp245_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=uqvqdiv_ssp245_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv partially regression onto north or south uqvqdiv"),
)




uq_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], uq_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], uq_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=uq_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq partially regression onto north or south uq"),
)
vq_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], vq_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], vq_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=vq_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq partially regression onto north or south vq"),
)
uqvqdiv_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], uqvqdiv_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], uqvqdiv_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=uqvqdiv_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv partially regression onto north or south uqvqdiv"),
)
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uq_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uq_his_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vq_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vq_his_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uqvqdiv_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uqvqdiv_his_north_south_partial_correlation.nc")

uq_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uq_his_north_south_partial_correlation.nc")
vq_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vq_his_north_south_partial_correlation.nc")
uqvqdiv_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uqvqdiv_his_north_south_partial_correlation.nc")


if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uq_ssp245_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uq_ssp245_p3_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vq_ssp245_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vq_ssp245_p3_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uqvqdiv_ssp245_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uqvqdiv_ssp245_p3_north_south_partial_correlation.nc")

uq_ssp245_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uq_ssp245_p3_north_south_partial_correlation.nc")
vq_ssp245_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vq_ssp245_p3_north_south_partial_correlation.nc")
uqvqdiv_ssp245_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uqvqdiv_ssp245_p3_north_south_partial_correlation.nc")




if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uq_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uq_ssp585_p3_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vq_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vq_ssp585_p3_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uqvqdiv_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uqvqdiv_ssp585_p3_north_south_partial_correlation.nc")

uq_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uq_ssp585_p3_north_south_partial_correlation.nc")
vq_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/vq_ssp585_p3_north_south_partial_correlation.nc")
uqvqdiv_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/uqvqdiv_ssp585_p3_north_south_partial_correlation.nc")

In [ ]:
# historical
pre_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], pre_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], pre_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=pre_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="precipitation partially regression onto north or south precipitation"),
)
hgt_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], hgt_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], hgt_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=hgt_his_north_south_slope.coords["models"].data,
        level=hgt_his_north_south_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt partially regression onto north or south hgt"),
)

u_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], u_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], u_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=u_his_north_south_slope.coords["models"].data,
        level=u_his_north_south_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u partially regression onto north or south u"),
)

v_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], v_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], v_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=v_his_north_south_slope.coords["models"].data,
        level=v_his_north_south_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v partially regression onto north or south v"),
)

# ssp245
pre_ssp245_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], pre_ssp245_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], pre_ssp245_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=pre_ssp245_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="precipitation partially regression onto north or south precipitation"),
)
hgt_ssp245_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], hgt_ssp245_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], hgt_ssp245_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=hgt_ssp245_p3_north_south_slope.coords["models"].data,
        level=hgt_ssp245_p3_north_south_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt partially regression onto north or south hgt"),
)

u_ssp245_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], u_ssp245_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], u_ssp245_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=u_ssp245_p3_north_south_slope.coords["models"].data,
        level=u_ssp245_p3_north_south_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u partially regression onto north or south u"),
)

v_ssp245_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], v_ssp245_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], v_ssp245_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=v_ssp245_p3_north_south_slope.coords["models"].data,
        level=v_ssp245_p3_north_south_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v partially regression onto north or south v"),
)

# ssp585
pre_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], pre_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], pre_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=pre_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="precipitation partially regression onto north or south precipitation"),
)
hgt_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], hgt_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], hgt_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=hgt_ssp585_p3_north_south_slope.coords["models"].data,
        level=hgt_ssp585_p3_north_south_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt partially regression onto north or south hgt"),
)

u_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], u_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], u_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=u_ssp585_p3_north_south_slope.coords["models"].data,
        level=u_ssp585_p3_north_south_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u partially regression onto north or south u"),
)

v_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], v_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], v_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=v_ssp585_p3_north_south_slope.coords["models"].data,
        level=v_ssp585_p3_north_south_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v partially regression onto north or south v"),
)


if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/pre_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/pre_his_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/hgt_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/hgt_his_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/u_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/u_his_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/v_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/v_his_north_south_partial_correlation.nc")

pre_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/pre_his_north_south_partial_correlation.nc")
hgt_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/hgt_his_north_south_partial_correlation.nc")
u_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/u_his_north_south_partial_correlation.nc")
v_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/v_his_north_south_partial_correlation.nc")


if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/pre_ssp245_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/pre_ssp245_p3_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/hgt_ssp245_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/hgt_ssp245_p3_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/u_ssp245_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/u_ssp245_p3_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/v_ssp245_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/v_ssp245_p3_north_south_partial_correlation.nc")

pre_ssp245_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/pre_ssp245_p3_north_south_partial_correlation.nc")
hgt_ssp245_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/hgt_ssp245_p3_north_south_partial_correlation.nc")
u_ssp245_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/u_ssp245_p3_north_south_partial_correlation.nc")
v_ssp245_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/v_ssp245_p3_north_south_partial_correlation.nc")





if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/pre_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/pre_ssp585_p3_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/hgt_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/hgt_ssp585_p3_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/u_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/u_ssp585_p3_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/v_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/v_ssp585_p3_north_south_partial_correlation.nc")

pre_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/pre_ssp585_p3_north_south_partial_correlation.nc")
hgt_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/hgt_ssp585_p3_north_south_partial_correlation.nc")
u_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/u_ssp585_p3_north_south_partial_correlation.nc")
v_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/v_ssp585_p3_north_south_partial_correlation.nc")

In [ ]:
# historical
sstDJF_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstDJF_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstDJF_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstDJF_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstDJF partially regression onto north or south sstDJF"),
)

sstMAM_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstMAM_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstMAM_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstMAM_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstMAM partially regression onto north or south sstMAM"),
)

sstJJA_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstJJA_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstJJA_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstJJA_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstJJA partially regression onto north or south sstJJA"),
)

sstSON_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstSON_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstSON_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstSON_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstSON partially regression onto north or south sstSON"),
)

sstnDJF_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstnDJF_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstnDJF_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstnDJF_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstnDJF partially regression onto north or south sstnDJF"),
)

# ssp585
sstDJF_ssp245_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstDJF_ssp245_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstDJF_ssp245_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstDJF_ssp245_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstDJF partially regression onto north or south sstDJF"),
)

sstMAM_ssp245_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstMAM_ssp245_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstMAM_ssp245_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstMAM_ssp245_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstMAM partially regression onto north or south sstMAM"),
)

sstJJA_ssp245_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstJJA_ssp245_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstJJA_ssp245_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstJJA_ssp245_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstJJA partially regression onto north or south sstJJA"),
)

sstSON_ssp245_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstSON_ssp245_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstSON_ssp245_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstSON_ssp245_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstSON partially regression onto north or south sstSON"),
)

sstnDJF_ssp245_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstnDJF_ssp245_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstnDJF_ssp245_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstnDJF_ssp245_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstnDJF partially regression onto north or south sstnDJF"),
)

# ssp585
sstDJF_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstDJF_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstDJF_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstDJF_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstDJF partially regression onto north or south sstDJF"),
)

sstMAM_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstMAM_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstMAM_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstMAM_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstMAM partially regression onto north or south sstMAM"),
)

sstJJA_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstJJA_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstJJA_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstJJA_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstJJA partially regression onto north or south sstJJA"),
)

sstSON_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstSON_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstSON_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstSON_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstSON partially regression onto north or south sstSON"),
)

sstnDJF_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstnDJF_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstnDJF_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstnDJF_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstnDJF partially regression onto north or south sstnDJF"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstDJF_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstDJF_his_north_south_partial_correlation.nc")
sstDJF_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstDJF_his_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstMAM_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstMAM_his_north_south_partial_correlation.nc")
sstMAM_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstMAM_his_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstJJA_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstJJA_his_north_south_partial_correlation.nc")
sstJJA_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstJJA_his_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstSON_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstSON_his_north_south_partial_correlation.nc")
sstSON_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstSON_his_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstnDJF_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstnDJF_his_north_south_partial_correlation.nc")
sstnDJF_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstnDJF_his_north_south_partial_correlation.nc")

# ssp585
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstDJF_ssp245_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstDJF_ssp245_p3_north_south_partial_correlation.nc")
sstDJF_ssp245_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstDJF_ssp245_p3_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstMAM_ssp245_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstMAM_ssp245_p3_north_south_partial_correlation.nc")
sstMAM_ssp245_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstMAM_ssp245_p3_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstJJA_ssp245_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstJJA_ssp245_p3_north_south_partial_correlation.nc")
sstJJA_ssp245_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstJJA_ssp245_p3_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstSON_ssp245_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstSON_ssp245_p3_north_south_partial_correlation.nc")
sstSON_ssp245_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstSON_ssp245_p3_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstnDJF_ssp245_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstnDJF_ssp245_p3_north_south_partial_correlation.nc")
sstnDJF_ssp245_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstnDJF_ssp245_p3_north_south_partial_correlation.nc")

# ssp585
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstDJF_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstDJF_ssp585_p3_north_south_partial_correlation.nc")
sstDJF_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstDJF_ssp585_p3_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstMAM_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstMAM_ssp585_p3_north_south_partial_correlation.nc")
sstMAM_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstMAM_ssp585_p3_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstJJA_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstJJA_ssp585_p3_north_south_partial_correlation.nc")
sstJJA_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstJJA_ssp585_p3_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstSON_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstSON_ssp585_p3_north_south_partial_correlation.nc")
sstSON_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstSON_ssp585_p3_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstnDJF_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstnDJF_ssp585_p3_north_south_partial_correlation.nc")
sstnDJF_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/sstnDJF_ssp585_p3_north_south_partial_correlation.nc")

# 计算模式表现

In [ ]:
hgt_his_north_south_slope = hgt_his_north_south_slope.transpose("area",...)

## 200hPa

In [ ]:
lon_ranking1_W = 50
lon_ranking1_E = 137.5
lat_ranking1_N = 47.5
lat_ranking1_S = 15.0
#   for 200hPa
lat_ranking_range1 = lat[(lat>=lat_ranking1_S) & (lat<=lat_ranking1_N)]
lon_ranking_range1 = lon[(lon>=lon_ranking1_W) & (lon<=lon_ranking1_E)]
IndR_200hgt_pcc = []
IndR_200hgt_RMSE = []
IndR_200hgt_std = []

for num_mod, mod in enumerate(models):
    
    hgt200_pcc = ca.cal_pcc(hgt_north_south_slope[0,:,:,:].sel(lat=lat_ranking_range1, lon=lon_ranking_range1, level=200.0), hgt_his_north_south_slope[0,:,:,:,:].sel(models=mod, lat=lat_ranking_range1, lon=lon_ranking_range1, level=200.0))
    
    IndR_200hgt_pcc.append(hgt200_pcc)

    IndR_200hgt_RMSE.append(np.sqrt(np.power((hgt_his_north_south_slope[0,:,:,:,:].sel(models=mod,lat=lat_ranking_range1,lon=lon_ranking_range1, level=200.0)-hgt_north_south_slope[0,:,:,:].sel(lat=lat_ranking_range1, lon=lon_ranking_range1, level=200.0)),2).mean(dim=["lat","lon"],skipna=True).data))
    
    IndR_200hgt_std.append(float((hgt_his_north_south_slope[0,:,:,:,:].sel(models=mod,lat=lat_ranking_range1,lon=lon_ranking_range1, level=200.0).std(dim=["lat","lon"],skipna=True)/hgt_north_south_slope[0,:,:,:].sel(lat=lat_ranking_range1, lon=lon_ranking_range1, level=200.0).std(dim=["lat","lon"],skipna=True)).data))



In [ ]:
models.where(np.array(IndR_200hgt_pcc)>=0.85)

## 850hPa

In [ ]:
lon_ranking2_W = 110
lon_ranking2_E = 137.5
lat_ranking2_N = 37.5
# lat_ranking2_N = 30
lat_ranking2_S = 15.0

#   for 850hPa
lat_ranking_range2 = lat[(lat>=lat_ranking2_S) & (lat<=lat_ranking2_N)]
lon_ranking_range2 = lon[(lon>=lon_ranking2_W) & (lon<=lon_ranking2_E)]
IndR_850hgt_pcc = []
IndR_850hgt_RMSE = []
IndR_850hgt_std = []

for num_mod, mod in enumerate(models):
    
    hgt850_pcc = ca.cal_pcc(hgt_north_south_slope[1,:,:,:].sel(lat=lat_ranking_range2, lon=lon_ranking_range2, level=850.0), hgt_his_north_south_slope[1,:,:,:,:].sel(models=mod, lat=lat_ranking_range2, lon=lon_ranking_range2, level=850.0))
    
    IndR_850hgt_pcc.append(hgt850_pcc)
    IndR_850hgt_RMSE.append(np.sqrt(np.power((hgt_his_north_south_slope[1,:,:,:,:].sel(models=mod,lat=lat_ranking_range2,lon=lon_ranking_range2, level=850.0)-hgt_north_south_slope[1,:,:,:].sel(lat=lat_ranking_range2, lon=lon_ranking_range2, level=850.0)),2).mean(dim=["lat","lon"],skipna=True).data))
    
    IndR_850hgt_std.append(float((hgt_his_north_south_slope[1,:,:,:,:].sel(models=mod,lat=lat_ranking_range2,lon=lon_ranking_range2, level=850.0).std(dim=["lat","lon"],skipna=True)/hgt_north_south_slope[1,:,:,:].sel(lat=lat_ranking_range2, lon=lon_ranking_range2, level=850.0).std(dim=["lat","lon"],skipna=True)).data))

In [ ]:
models.where(np.array(IndR_850hgt_pcc)>=0.85)

# 计算各要素回归到印度降水的结果

## historical

In [ ]:
level = uhis_ver_JJA.level
sel_level = level[level>=100.0]
(
    IndR_his_uJJA_slope,
    IndR_his_uJJA_intercept,
    IndR_his_uJJA_rvalue,
    IndR_his_uJJA_pvalue,
    IndR_his_uJJA_hypothesis,
) = ca.dim_linregress(prehis_India_JJA_nor, uhis_ver_JJA.sel(level=sel_level))

(
    IndR_his_vJJA_slope,
    IndR_his_vJJA_intercept,
    IndR_his_vJJA_rvalue,
    IndR_his_vJJA_pvalue,
    IndR_his_vJJA_hypothesis,
) = ca.dim_linregress(prehis_India_JJA_nor, vhis_ver_JJA.sel(level=sel_level))

(
    IndR_his_q_slope,
    IndR_his_q_intercept,
    IndR_his_q_rvalue,
    IndR_his_q_pvalue,
    IndR_his_q_hypothesis,
) = ca.dim_linregress(prehis_India_JJA_nor, qhis_ver_JJA.sel(level=sel_level))

(
    IndR_his_wap_slope,
    IndR_his_wap_intercept,
    IndR_his_wap_rvalue,
    IndR_his_wap_pvalue,
    IndR_his_wap_hypothesis,
) = ca.dim_linregress(prehis_India_JJA_nor, waphis_ver_JJA.sel(level=sel_level))

(
    IndR_his_hfls_slope,
    IndR_his_hfls_intercept,
    IndR_his_hfls_rvalue,
    IndR_his_hfls_pvalue,
    IndR_his_hfls_hypothesis,
) = ca.dim_linregress(prehis_India_JJA_nor, hflshis_JJA)


(
    IndR_his_sp_slope,
    IndR_his_sp_intercept,
    IndR_his_sp_rvalue,
    IndR_his_sp_pvalue,
    IndR_his_sp_hypothesis,
) = ca.dim_linregress(prehis_India_JJA_nor, sphis_JJA)

In [ ]:
IndR_his_uJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level",  "lat", "lon"], IndR_his_uJJA_slope.data),
        intercept=(["models", "level",  "lat", "lon"], IndR_his_uJJA_intercept.data),
        rvalue=(["models", "level",  "lat", "lon"], IndR_his_uJJA_rvalue.data),
        pvalue=(["models", "level",  "lat", "lon"], IndR_his_uJJA_pvalue.data),
        hypothesis=(["models", "level",  "lat", "lon"], IndR_his_uJJA_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=sel_level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uJJA fields regress"),
)
ca.save(IndR_his_uJJA_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_uJJA_regress.nc")

IndR_his_vJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level",  "lat", "lon"], IndR_his_vJJA_slope.data),
        intercept=(["models", "level",  "lat", "lon"], IndR_his_vJJA_intercept.data),
        rvalue=(["models", "level",  "lat", "lon"], IndR_his_vJJA_rvalue.data),
        pvalue=(["models", "level",  "lat", "lon"], IndR_his_vJJA_pvalue.data),
        hypothesis=(["models", "level",  "lat", "lon"], IndR_his_vJJA_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=sel_level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vJJA fields regress"),
)
ca.save(IndR_his_vJJA_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_vJJA_regress.nc")

IndR_his_q_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level",  "lat", "lon"], IndR_his_q_slope.data),
        intercept=(["models", "level",  "lat", "lon"], IndR_his_q_intercept.data),
        rvalue=(["models", "level",  "lat", "lon"], IndR_his_q_rvalue.data),
        pvalue=(["models", "level",  "lat", "lon"], IndR_his_q_pvalue.data),
        hypothesis=(["models", "level",  "lat", "lon"], IndR_his_q_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=sel_level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="q fields regress"),
)
ca.save(IndR_his_q_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_q_regress.nc")


IndR_his_wap_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level",  "lat", "lon"], IndR_his_wap_slope.data),
        intercept=(["models", "level",  "lat", "lon"], IndR_his_wap_intercept.data),
        rvalue=(["models", "level",  "lat", "lon"], IndR_his_wap_rvalue.data),
        pvalue=(["models", "level",  "lat", "lon"], IndR_his_wap_pvalue.data),
        hypothesis=(["models", "level",  "lat", "lon"], IndR_his_wap_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=sel_level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="wap fields regress"),
)
ca.save(IndR_his_wap_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_wap_regress.nc")

IndR_his_hfls_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_hfls_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_hfls_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_hfls_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_hfls_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_hfls_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hfls fields regress"),
)
ca.save(IndR_his_hfls_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_hfls_regress.nc")

IndR_his_sp_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_sp_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_sp_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_sp_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_sp_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_sp_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sp fields regress"),
)
ca.save(IndR_his_sp_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_sp_regress.nc")

## ssp245_p3

In [ ]:
(
    IndR_ssp245_p3_uJJA_slope,
    IndR_ssp245_p3_uJJA_intercept,
    IndR_ssp245_p3_uJJA_rvalue,
    IndR_ssp245_p3_uJJA_pvalue,
    IndR_ssp245_p3_uJJA_hypothesis,
) = ca.dim_linregress(pressp245_p3_India_JJA_nor, ussp245_p3_ver_JJA.sel(level=sel_level))

(
    IndR_ssp245_p3_vJJA_slope,
    IndR_ssp245_p3_vJJA_intercept,
    IndR_ssp245_p3_vJJA_rvalue,
    IndR_ssp245_p3_vJJA_pvalue,
    IndR_ssp245_p3_vJJA_hypothesis,
) = ca.dim_linregress(pressp245_p3_India_JJA_nor, vssp245_p3_ver_JJA.sel(level=sel_level))

(
    IndR_ssp245_p3_q_slope,
    IndR_ssp245_p3_q_intercept,
    IndR_ssp245_p3_q_rvalue,
    IndR_ssp245_p3_q_pvalue,
    IndR_ssp245_p3_q_hypothesis,
) = ca.dim_linregress(pressp245_p3_India_JJA_nor, qssp245_p3_ver_JJA.sel(level=sel_level))

(
    IndR_ssp245_p3_wap_slope,
    IndR_ssp245_p3_wap_intercept,
    IndR_ssp245_p3_wap_rvalue,
    IndR_ssp245_p3_wap_pvalue,
    IndR_ssp245_p3_wap_hypothesis,
) = ca.dim_linregress(pressp245_p3_India_JJA_nor, wapssp245_p3_ver_JJA.sel(level=sel_level))

(
    IndR_ssp245_p3_hfls_slope,
    IndR_ssp245_p3_hfls_intercept,
    IndR_ssp245_p3_hfls_rvalue,
    IndR_ssp245_p3_hfls_pvalue,
    IndR_ssp245_p3_hfls_hypothesis,
) = ca.dim_linregress(pressp245_p3_India_JJA_nor, hflsssp245_p3_ver_JJA)


(
    IndR_ssp245_p3_sp_slope,
    IndR_ssp245_p3_sp_intercept,
    IndR_ssp245_p3_sp_rvalue,
    IndR_ssp245_p3_sp_pvalue,
    IndR_ssp245_p3_sp_hypothesis,
) = ca.dim_linregress(pressp245_p3_India_JJA_nor, spssp245_p3_JJA)

In [ ]:
IndR_ssp245_p3_uJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_uJJA_slope.data),
        intercept=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_uJJA_intercept.data),
        rvalue=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_uJJA_rvalue.data),
        pvalue=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_uJJA_pvalue.data),
        hypothesis=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_uJJA_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=sel_level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uJJA fields regress"),
)
ca.save(IndR_ssp245_p3_uJJA_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_uJJA_regress.nc")

IndR_ssp245_p3_vJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_vJJA_slope.data),
        intercept=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_vJJA_intercept.data),
        rvalue=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_vJJA_rvalue.data),
        pvalue=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_vJJA_pvalue.data),
        hypothesis=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_vJJA_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=sel_level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vJJA fields regress"),
)
ca.save(IndR_ssp245_p3_vJJA_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_vJJA_regress.nc")

IndR_ssp245_p3_q_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_q_slope.data),
        intercept=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_q_intercept.data),
        rvalue=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_q_rvalue.data),
        pvalue=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_q_pvalue.data),
        hypothesis=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_q_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=sel_level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="q fields regress"),
)
ca.save(IndR_ssp245_p3_q_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_q_regress.nc")


IndR_ssp245_p3_wap_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_wap_slope.data),
        intercept=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_wap_intercept.data),
        rvalue=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_wap_rvalue.data),
        pvalue=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_wap_pvalue.data),
        hypothesis=(["models", "level",  "lat", "lon"], IndR_ssp245_p3_wap_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=sel_level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="wap fields regress"),
)
ca.save(IndR_ssp245_p3_wap_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_wap_regress.nc")

IndR_ssp245_p3_hfls_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp245_p3_hfls_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp245_p3_hfls_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp245_p3_hfls_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp245_p3_hfls_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp245_p3_hfls_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hfls fields regress"),
)
ca.save(IndR_ssp245_p3_hfls_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_hfls_regress.nc")

IndR_ssp245_p3_sp_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp245_p3_sp_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp245_p3_sp_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp245_p3_sp_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp245_p3_sp_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp245_p3_sp_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sp fields regress"),
)
ca.save(IndR_ssp245_p3_sp_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_sp_regress.nc")

## ssp585_p3

In [ ]:
(
    IndR_ssp585_p3_uJJA_slope,
    IndR_ssp585_p3_uJJA_intercept,
    IndR_ssp585_p3_uJJA_rvalue,
    IndR_ssp585_p3_uJJA_pvalue,
    IndR_ssp585_p3_uJJA_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, ussp585_p3_ver_JJA.sel(level=sel_level))

(
    IndR_ssp585_p3_vJJA_slope,
    IndR_ssp585_p3_vJJA_intercept,
    IndR_ssp585_p3_vJJA_rvalue,
    IndR_ssp585_p3_vJJA_pvalue,
    IndR_ssp585_p3_vJJA_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, vssp585_p3_ver_JJA.sel(level=sel_level))

(
    IndR_ssp585_p3_q_slope,
    IndR_ssp585_p3_q_intercept,
    IndR_ssp585_p3_q_rvalue,
    IndR_ssp585_p3_q_pvalue,
    IndR_ssp585_p3_q_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, qssp585_p3_ver_JJA.sel(level=sel_level))

(
    IndR_ssp585_p3_wap_slope,
    IndR_ssp585_p3_wap_intercept,
    IndR_ssp585_p3_wap_rvalue,
    IndR_ssp585_p3_wap_pvalue,
    IndR_ssp585_p3_wap_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, wapssp585_p3_ver_JJA.sel(level=sel_level))

(
    IndR_ssp585_p3_hfls_slope,
    IndR_ssp585_p3_hfls_intercept,
    IndR_ssp585_p3_hfls_rvalue,
    IndR_ssp585_p3_hfls_pvalue,
    IndR_ssp585_p3_hfls_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, hflsssp585_p3_JJA)


(
    IndR_ssp585_p3_sp_slope,
    IndR_ssp585_p3_sp_intercept,
    IndR_ssp585_p3_sp_rvalue,
    IndR_ssp585_p3_sp_pvalue,
    IndR_ssp585_p3_sp_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, spssp585_p3_JJA)

In [ ]:
IndR_ssp585_p3_uJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_uJJA_slope.data),
        intercept=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_uJJA_intercept.data),
        rvalue=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_uJJA_rvalue.data),
        pvalue=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_uJJA_pvalue.data),
        hypothesis=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_uJJA_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=sel_level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uJJA fields regress"),
)
ca.save(IndR_ssp585_p3_uJJA_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_uJJA_regress.nc")

IndR_ssp585_p3_vJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_vJJA_slope.data),
        intercept=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_vJJA_intercept.data),
        rvalue=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_vJJA_rvalue.data),
        pvalue=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_vJJA_pvalue.data),
        hypothesis=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_vJJA_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=sel_level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vJJA fields regress"),
)
ca.save(IndR_ssp585_p3_vJJA_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_vJJA_regress.nc")

IndR_ssp585_p3_q_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_q_slope.data),
        intercept=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_q_intercept.data),
        rvalue=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_q_rvalue.data),
        pvalue=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_q_pvalue.data),
        hypothesis=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_q_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=sel_level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="q fields regress"),
)
ca.save(IndR_ssp585_p3_q_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_q_regress.nc")


IndR_ssp585_p3_wap_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_wap_slope.data),
        intercept=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_wap_intercept.data),
        rvalue=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_wap_rvalue.data),
        pvalue=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_wap_pvalue.data),
        hypothesis=(["models", "level",  "lat", "lon"], IndR_ssp585_p3_wap_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=sel_level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="wap fields regress"),
)
ca.save(IndR_ssp585_p3_wap_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_wap_regress.nc")

IndR_ssp585_p3_hfls_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_hfls_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_hfls_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_hfls_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_hfls_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_hfls_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hfls fields regress"),
)
ca.save(IndR_ssp585_p3_hfls_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_hfls_regress.nc")

IndR_ssp585_p3_sp_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_sp_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_sp_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_sp_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_sp_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_sp_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sp fields regress"),
)
ca.save(IndR_ssp585_p3_sp_regress, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_sp_regress.nc")

# 读取回归数据

In [10]:
IndR_his_pre_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_pre_regress.nc")
IndR_his_uJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_uJJA_regress.nc")
IndR_his_vJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_vJJA_regress.nc")
IndR_his_q_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_q_regress.nc")
IndR_his_wap_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_wap_regress.nc")
IndR_his_hfls_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_hfls_regress.nc")
IndR_his_sp_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_his_sp_regress.nc")

IndR_ssp245_p3_pre_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_pre_regress.nc")
IndR_ssp245_p3_uJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_uJJA_regress.nc")
IndR_ssp245_p3_vJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_vJJA_regress.nc")
IndR_ssp245_p3_q_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_q_regress.nc")
IndR_ssp245_p3_wap_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_wap_regress.nc")
IndR_ssp245_p3_hfls_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_hfls_regress.nc")
IndR_ssp245_p3_sp_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp245_p3_sp_regress.nc")



IndR_ssp585_p3_pre_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_pre_regress.nc")
IndR_ssp585_p3_uJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_uJJA_regress.nc")
IndR_ssp585_p3_vJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_vJJA_regress.nc")
IndR_ssp585_p3_q_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_q_regress.nc")
IndR_ssp585_p3_wap_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_wap_regress.nc")
IndR_ssp585_p3_hfls_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_hfls_regress.nc")
IndR_ssp585_p3_sp_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/IndR_ssp585_p3_sp_regress.nc")



In [11]:
IndR_his_pre_slope = IndR_his_pre_regress["slope"]
IndR_his_pre_rvalue = IndR_his_pre_regress["rvalue"]
IndR_his_pre_pvalue = IndR_his_pre_regress["pvalue"]

IndR_his_uJJA_slope = IndR_his_uJJA_regress["slope"]
IndR_his_uJJA_rvalue = IndR_his_uJJA_regress["rvalue"]
IndR_his_uJJA_pvalue = IndR_his_uJJA_regress["pvalue"]

IndR_his_vJJA_slope = IndR_his_vJJA_regress["slope"]
IndR_his_vJJA_rvalue = IndR_his_vJJA_regress["rvalue"]
IndR_his_vJJA_pvalue = IndR_his_vJJA_regress["pvalue"]

IndR_his_q_slope = IndR_his_q_regress["slope"]
IndR_his_q_rvalue = IndR_his_q_regress["rvalue"]
IndR_his_q_pvalue = IndR_his_q_regress["pvalue"]

IndR_his_wap_slope = IndR_his_wap_regress["slope"]
IndR_his_wap_rvalue = IndR_his_wap_regress["rvalue"]
IndR_his_wap_pvalue = IndR_his_wap_regress["pvalue"]

IndR_his_hfls_slope = IndR_his_hfls_regress["slope"]
IndR_his_hfls_rvalue = IndR_his_hfls_regress["rvalue"]
IndR_his_hfls_pvalue = IndR_his_hfls_regress["pvalue"]

IndR_his_sp_slope = IndR_his_sp_regress["slope"]
IndR_his_sp_rvalue = IndR_his_sp_regress["rvalue"]
IndR_his_sp_pvalue = IndR_his_sp_regress["pvalue"]


In [12]:
# ssp245_p3

IndR_ssp245_p3_pre_slope = IndR_ssp245_p3_pre_regress["slope"]
IndR_ssp245_p3_pre_rvalue = IndR_ssp245_p3_pre_regress["rvalue"]
IndR_ssp245_p3_pre_pvalue = IndR_ssp245_p3_pre_regress["pvalue"]

IndR_ssp245_p3_uJJA_slope = IndR_ssp245_p3_uJJA_regress["slope"]
IndR_ssp245_p3_uJJA_rvalue = IndR_ssp245_p3_uJJA_regress["rvalue"]
IndR_ssp245_p3_uJJA_pvalue = IndR_ssp245_p3_uJJA_regress["pvalue"]

IndR_ssp245_p3_vJJA_slope = IndR_ssp245_p3_vJJA_regress["slope"]
IndR_ssp245_p3_vJJA_rvalue = IndR_ssp245_p3_vJJA_regress["rvalue"]
IndR_ssp245_p3_vJJA_pvalue = IndR_ssp245_p3_vJJA_regress["pvalue"]

IndR_ssp245_p3_q_slope = IndR_ssp245_p3_q_regress["slope"]
IndR_ssp245_p3_q_rvalue = IndR_ssp245_p3_q_regress["rvalue"]
IndR_ssp245_p3_q_pvalue = IndR_ssp245_p3_q_regress["pvalue"]

IndR_ssp245_p3_wap_slope = IndR_ssp245_p3_wap_regress["slope"]
IndR_ssp245_p3_wap_rvalue = IndR_ssp245_p3_wap_regress["rvalue"]
IndR_ssp245_p3_wap_pvalue = IndR_ssp245_p3_wap_regress["pvalue"]

IndR_ssp245_p3_hfls_slope = IndR_ssp245_p3_hfls_regress["slope"]
IndR_ssp245_p3_hfls_rvalue = IndR_ssp245_p3_hfls_regress["rvalue"]
IndR_ssp245_p3_hfls_pvalue = IndR_ssp245_p3_hfls_regress["pvalue"]

IndR_ssp245_p3_sp_slope = IndR_ssp245_p3_sp_regress["slope"]
IndR_ssp245_p3_sp_rvalue = IndR_ssp245_p3_sp_regress["rvalue"]
IndR_ssp245_p3_sp_pvalue = IndR_ssp245_p3_sp_regress["pvalue"]

In [13]:
# ssp585_p3

IndR_ssp585_p3_pre_slope = IndR_ssp585_p3_pre_regress["slope"]
IndR_ssp585_p3_pre_rvalue = IndR_ssp585_p3_pre_regress["rvalue"]
IndR_ssp585_p3_pre_pvalue = IndR_ssp585_p3_pre_regress["pvalue"]

IndR_ssp585_p3_uJJA_slope = IndR_ssp585_p3_uJJA_regress["slope"]
IndR_ssp585_p3_uJJA_rvalue = IndR_ssp585_p3_uJJA_regress["rvalue"]
IndR_ssp585_p3_uJJA_pvalue = IndR_ssp585_p3_uJJA_regress["pvalue"]

IndR_ssp585_p3_vJJA_slope = IndR_ssp585_p3_vJJA_regress["slope"]
IndR_ssp585_p3_vJJA_rvalue = IndR_ssp585_p3_vJJA_regress["rvalue"]
IndR_ssp585_p3_vJJA_pvalue = IndR_ssp585_p3_vJJA_regress["pvalue"]

IndR_ssp585_p3_q_slope = IndR_ssp585_p3_q_regress["slope"]
IndR_ssp585_p3_q_rvalue = IndR_ssp585_p3_q_regress["rvalue"]
IndR_ssp585_p3_q_pvalue = IndR_ssp585_p3_q_regress["pvalue"]

IndR_ssp585_p3_wap_slope = IndR_ssp585_p3_wap_regress["slope"]
IndR_ssp585_p3_wap_rvalue = IndR_ssp585_p3_wap_regress["rvalue"]
IndR_ssp585_p3_wap_pvalue = IndR_ssp585_p3_wap_regress["pvalue"]

IndR_ssp585_p3_hfls_slope = IndR_ssp585_p3_hfls_regress["slope"]
IndR_ssp585_p3_hfls_rvalue = IndR_ssp585_p3_hfls_regress["rvalue"]
IndR_ssp585_p3_hfls_pvalue = IndR_ssp585_p3_hfls_regress["pvalue"]

IndR_ssp585_p3_sp_slope = IndR_ssp585_p3_sp_regress["slope"]
IndR_ssp585_p3_sp_rvalue = IndR_ssp585_p3_sp_regress["rvalue"]
IndR_ssp585_p3_sp_pvalue = IndR_ssp585_p3_sp_regress["pvalue"]

# 计算气候态平均

In [14]:
#historical
pre_his = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/pr_historical_r144x72_195001-201412.nc")
u_his = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/ua_historical_r144x72_195001-201412.nc")
v_his = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/va_historical_r144x72_195001-201412.nc")
hus_his = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/hus_historical_r144x72_195001-201412.nc")
wap_his = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/wap_historical_r144x72_197901-201412.nc")
ps_his = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/sp_historical_r144x72_197901-201412.nc")

In [15]:
pre_his = pre_his.sel(time=pre_his.time.dt.year>=1979)
u_his = u_his.sel(time=u_his.time.dt.year>=1979)
v_his = v_his.sel(time=v_his.time.dt.year>=1979)
hus_his = hus_his.sel(time=hus_his.time.dt.year>=1979)
wap_his = wap_his.sel(time=wap_his.time.dt.year>=1979)
ps_his = ps_his.sel(time=ps_his.time.dt.year>=1979)

In [16]:
pre_his_bar = pre_his.mean(dim="time",skipna=True)
u_his_bar = u_his.mean(dim="time",skipna=True)
v_his_bar = v_his.mean(dim="time",skipna=True)
hus_his_bar = hus_his.mean(dim="time",skipna=True)
wap_his_bar = wap_his.mean(dim="time",skipna=True)
ps_his_bar = ps_his.mean(dim="time",skipna=True)

In [17]:
# ssp245_p3
pre_ssp245_p3 = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/pr_ssp245_r144x72_201501-209912.nc")
u_ssp245_p3 = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/ua_ssp245_r144x72_201501-209912.nc")
v_ssp245_p3 = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/va_ssp245_r144x72_201501-209912.nc")
hus_ssp245_p3 = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/hus_ssp245_r144x72_201501-209912.nc")
wap_ssp245_p3 = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/wap_ssp245_r144x72_201501-209912.nc")
ps_ssp245_p3 = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/ps_ssp245_r144x72_201501-209912.nc")

In [18]:
pre_ssp245_p3 = pre_ssp245_p3.sel(time=pre_ssp245_p3.time.dt.year>=2064)
u_ssp245_p3 = u_ssp245_p3.sel(time=u_ssp245_p3.time.dt.year>=2064)
v_ssp245_p3 = v_ssp245_p3.sel(time=v_ssp245_p3.time.dt.year>=2064)
hus_ssp245_p3 = hus_ssp245_p3.sel(time=hus_ssp245_p3.time.dt.year>=2064)
wap_ssp245_p3 = wap_ssp245_p3.sel(time=wap_ssp245_p3.time.dt.year>=2064)
ps_ssp245_p3 = ps_ssp245_p3.sel(time=ps_ssp245_p3.time.dt.year>=2064)

In [19]:
pre_ssp245_p3_bar = pre_ssp245_p3.mean(dim="time")
u_ssp245_p3_bar = u_ssp245_p3.mean(dim="time")
v_ssp245_p3_bar = v_ssp245_p3.mean(dim="time")
hus_ssp245_p3_bar = hus_ssp245_p3.mean(dim="time")
wap_ssp245_p3_bar = wap_ssp245_p3.mean(dim="time")
ps_ssp245_p3_bar = ps_ssp245_p3.mean(dim="time")

In [20]:
# ssp585_p3
pre_ssp585_p3 = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/pr_ssp585_r144x72_201501-209912.nc")
u_ssp585_p3 = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/ua_ssp585_r144x72_201501-209912.nc")
v_ssp585_p3 = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/va_ssp585_r144x72_201501-209912.nc")
hus_ssp585_p3 = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/hus_ssp585_r144x72_201501-209912.nc")
wap_ssp585_p3 = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/wap_ssp585_r144x72_201501-209912.nc")
ps_ssp585_p3 = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/ps_ssp585_r144x72_201501-209912.nc")

In [21]:
pre_ssp585_p3 = pre_ssp585_p3.sel(time=pre_ssp585_p3.time.dt.year>=2064)
u_ssp585_p3 = u_ssp585_p3.sel(time=u_ssp585_p3.time.dt.year>=2064)
v_ssp585_p3 = v_ssp585_p3.sel(time=v_ssp585_p3.time.dt.year>=2064)
hus_ssp585_p3 = hus_ssp585_p3.sel(time=hus_ssp585_p3.time.dt.year>=2064)
wap_ssp585_p3 = wap_ssp585_p3.sel(time=wap_ssp585_p3.time.dt.year>=2064)
ps_ssp585_p3 = ps_ssp585_p3.sel(time=ps_ssp585_p3.time.dt.year>=2064)

In [22]:
pre_ssp585_p3_bar = pre_ssp585_p3.mean(dim="time")
u_ssp585_p3_bar = u_ssp585_p3.mean(dim="time")
v_ssp585_p3_bar = v_ssp585_p3.mean(dim="time")
hus_ssp585_p3_bar = hus_ssp585_p3.mean(dim="time")
wap_ssp585_p3_bar = wap_ssp585_p3.mean(dim="time")
ps_ssp585_p3_bar = ps_ssp585_p3.mean(dim="time")

# 水汽诊断方程

In [23]:
level = uhis_ver_JJA.level
sel_level = level[level>=100.0]

def recoordinate(var, level,  lat, lon):
  var.coords["level"] = level
  var.coords["lat"] = lat
  var.coords["lon"] = lon
  return(var)

def create_dpdydx(models, sel_level, lat, lon):
  tmp_dp_array, tmp_dx_array, tmp_dy_array = np.zeros((len(models), len(sel_level), len(lat), len(lon))), np.zeros((len(models), len(sel_level), len(lat), len(lon))), np.zeros((len(models), len(sel_level), len(lat), len(lon)))
  tmp_dp = xr.DataArray(
  data=tmp_dp_array, 
  dims=["models", "level", "lat", "lon"],
  coords=dict(
    models=models.data,
    level=sel_level.data,
    lat=lat.data,
    lon=lon.data
    )
  )

  tmp_dy = xr.DataArray(
    data=tmp_dy_array, 
    dims=["models", "level", "lat", "lon"],
    coords=dict(
      models=models.data,
      level=sel_level.data,
      lat=lat.data,
      lon=lon.data
    )
  )

  tmp_dx = xr.DataArray(
    data=tmp_dx_array, 
    dims=["models", "level", "lat", "lon"],
    coords=dict(
      models=models.data,
      level=sel_level.data,
      lat=lat.data,
      lon=lon.data
    )
  )
  return(tmp_dp, tmp_dy, tmp_dx)

## 计算水汽的偏微分

In [24]:
hus_his_prime_dp, hus_his_prime_dy, hus_his_prime_dx = create_dpdydx(models, sel_level, lat, lon)
hus_his_bar_dp, hus_his_bar_dy, hus_his_bar_dx = create_dpdydx(models, sel_level, lat, lon)

hus_ssp245_p3_prime_dp, hus_ssp245_p3_prime_dy, hus_ssp245_p3_prime_dx = create_dpdydx(models, sel_level, lat, lon)
hus_ssp245_p3_bar_dp, hus_ssp245_p3_bar_dy, hus_ssp245_p3_bar_dx = create_dpdydx(models, sel_level, lat, lon)

hus_ssp585_p3_prime_dp, hus_ssp585_p3_prime_dy, hus_ssp585_p3_prime_dx = create_dpdydx(models, sel_level, lat, lon)
hus_ssp585_p3_bar_dp, hus_ssp585_p3_bar_dy, hus_ssp585_p3_bar_dx = create_dpdydx(models, sel_level, lat, lon)

for num_mod, mod in enumerate(models):
  hus_his_prime_dp[num_mod], hus_his_prime_dy[num_mod], hus_his_prime_dx[num_mod] = mpcalc.gradient(recoordinate(IndR_his_q_slope, sel_level, lat, lon).sel(models=mod))
  hus_his_bar_dp[num_mod], hus_his_bar_dy[num_mod], hus_his_bar_dx[num_mod] = mpcalc.gradient(hus_his_bar.sel(level=sel_level).sel(models=mod))
  
  hus_ssp245_p3_prime_dp[num_mod], hus_ssp245_p3_prime_dy[num_mod], hus_ssp245_p3_prime_dx[num_mod] = mpcalc.gradient(recoordinate(IndR_ssp245_p3_q_slope, sel_level, lat, lon).sel(models=mod))
  hus_ssp245_p3_bar_dp[num_mod], hus_ssp245_p3_bar_dy[num_mod], hus_ssp245_p3_bar_dx[num_mod] = mpcalc.gradient(hus_ssp245_p3_bar.sel(level=sel_level).sel(models=mod))
  
  hus_ssp585_p3_prime_dp[num_mod], hus_ssp585_p3_prime_dy[num_mod], hus_ssp585_p3_prime_dx[num_mod] = mpcalc.gradient(recoordinate(IndR_ssp585_p3_q_slope, sel_level, lat, lon).sel(models=mod))
  hus_ssp585_p3_bar_dp[num_mod], hus_ssp585_p3_bar_dy[num_mod], hus_ssp585_p3_bar_dx[num_mod] = mpcalc.gradient(hus_ssp585_p3_bar.sel(level=sel_level).sel(models=mod))

In [25]:
ptop = 100
g = 9.8
cal_level = hus_his_bar.coords["level"].where(hus_his_bar.coords["level"] >= ptop, drop=True)*100
cal_level.attrs["units"] = "Pa"

hisdp = geocat.comp.dpres_plevel(cal_level, ps_his_bar, ptop*100)
hisdpg = hisdp / g
hisdpg.attrs["units"] = "kg/m2"

ssp245_p3dp = geocat.comp.dpres_plevel(cal_level, ps_ssp245_p3_bar, ptop*100)
ssp245_p3dpg = ssp245_p3dp / g
ssp245_p3dpg.attrs["units"] = "kg/m2"

ssp585_p3dp = geocat.comp.dpres_plevel(cal_level, ps_ssp585_p3_bar, ptop*100)
ssp585_p3dpg = ssp585_p3dp / g
ssp585_p3dpg.attrs["units"] = "kg/m2"

integration_top = 1

def vert_intergrate(da_sl, CMIP6dpg, integration_top=integration_top, cal_level=cal_level):
  res_vert = (da_sl * CMIP6dpg.data).sum(dim="level", skipna=True)
  return(res_vert)

## historical

In [35]:
term_his_1_1 = vert_intergrate(u_his_bar.sel(level=sel_level)*hus_his_prime_dx.sel(level=sel_level).data, CMIP6dpg=hisdpg)
term_his_1_2 = vert_intergrate(v_his_bar.sel(level=sel_level)*hus_his_prime_dy.sel(level=sel_level).data, CMIP6dpg=hisdpg)

term_his_1 = term_his_1_1 + term_his_1_2

term_his_2_1 = vert_intergrate(IndR_his_uJJA_slope.sel(level=sel_level)*hus_his_bar_dx.sel(level=sel_level).data, CMIP6dpg=hisdpg)
term_his_2_2 = vert_intergrate(IndR_his_vJJA_slope.sel(level=sel_level)*hus_his_bar_dy.sel(level=sel_level).data, CMIP6dpg=hisdpg)
term_his_2 = term_his_2_1+term_his_2_2

term_his_3 = vert_intergrate(wap_his_bar.sel(level=sel_level)*hus_his_prime_dp.data, CMIP6dpg=hisdpg)

term_his_4 = vert_intergrate(IndR_his_wap_slope.sel(level=sel_level)*hus_his_bar_dp.data, CMIP6dpg=hisdpg)

term_his_NLRes = IndR_his_pre_slope-IndR_his_hfls_slope+term_his_1+term_his_2+term_his_3+term_his_4

In [36]:
ca.save(term_his_1, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_his_1")
ca.save(term_his_2, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_his_2")
ca.save(term_his_3, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_his_3")
ca.save(term_his_4, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_his_4")
ca.save(term_his_NLRes, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_his_NLRes")

## ssp245_p3

In [37]:
term_ssp245_p3_1_1 = vert_intergrate(u_ssp245_p3_bar.sel(level=sel_level)*hus_ssp245_p3_prime_dx.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_ssp245_p3_1_2 = vert_intergrate(v_ssp245_p3_bar.sel(level=sel_level)*hus_ssp245_p3_prime_dy.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)

term_ssp245_p3_1 = term_ssp245_p3_1_1 + term_ssp245_p3_1_2

term_ssp245_p3_2_1 = vert_intergrate(IndR_ssp245_p3_uJJA_slope.sel(level=sel_level)*hus_ssp245_p3_bar_dx.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_ssp245_p3_2_2 = vert_intergrate(IndR_ssp245_p3_vJJA_slope.sel(level=sel_level)*hus_ssp245_p3_bar_dy.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_ssp245_p3_2 = term_ssp245_p3_2_1+term_ssp245_p3_2_2

term_ssp245_p3_3 = vert_intergrate(wap_ssp245_p3_bar.sel(level=sel_level)*hus_ssp245_p3_prime_dp.data, CMIP6dpg=ssp245_p3dpg)

term_ssp245_p3_4 = vert_intergrate(IndR_ssp245_p3_wap_slope.sel(level=sel_level)*hus_ssp245_p3_bar_dp.data, CMIP6dpg=ssp245_p3dpg)

term_ssp245_p3_NLRes = IndR_ssp245_p3_pre_slope-IndR_ssp245_p3_hfls_slope+term_ssp245_p3_1+term_ssp245_p3_2+term_ssp245_p3_3+term_ssp245_p3_4

In [38]:
ca.save(term_ssp245_p3_1, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ssp245_p3_1")
ca.save(term_ssp245_p3_2, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ssp245_p3_2")
ca.save(term_ssp245_p3_3, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ssp245_p3_3")
ca.save(term_ssp245_p3_4, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ssp245_p3_4")
ca.save(term_ssp245_p3_NLRes, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ssp245_p3_NLRes")

## ssp585_p3

In [39]:
term_ssp585_p3_1_1 = vert_intergrate(u_ssp585_p3_bar.sel(level=sel_level)*hus_ssp585_p3_prime_dx.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_ssp585_p3_1_2 = vert_intergrate(v_ssp585_p3_bar.sel(level=sel_level)*hus_ssp585_p3_prime_dy.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)

term_ssp585_p3_1 = term_ssp585_p3_1_1 + term_ssp585_p3_1_2

term_ssp585_p3_2_1 = vert_intergrate(IndR_ssp585_p3_uJJA_slope.sel(level=sel_level)*hus_ssp585_p3_bar_dx.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_ssp585_p3_2_2 = vert_intergrate(IndR_ssp585_p3_vJJA_slope.sel(level=sel_level)*hus_ssp585_p3_bar_dy.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_ssp585_p3_2 = term_ssp585_p3_2_1+term_ssp585_p3_2_2

term_ssp585_p3_3 = vert_intergrate(wap_ssp585_p3_bar.sel(level=sel_level)*hus_ssp585_p3_prime_dp.data, CMIP6dpg=ssp585_p3dpg)

term_ssp585_p3_4 = vert_intergrate(IndR_ssp585_p3_wap_slope.sel(level=sel_level)*hus_ssp585_p3_bar_dp.data, CMIP6dpg=ssp585_p3dpg)

term_ssp585_p3_NLRes = IndR_ssp585_p3_pre_slope-IndR_ssp585_p3_hfls_slope+term_ssp585_p3_1+term_ssp585_p3_2+term_ssp585_p3_3+term_ssp585_p3_4

In [40]:
ca.save(term_ssp585_p3_1, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ssp585_p3_1")
ca.save(term_ssp585_p3_2, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ssp585_p3_2")
ca.save(term_ssp585_p3_3, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ssp585_p3_3")
ca.save(term_ssp585_p3_4, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ssp585_p3_4")
ca.save(term_ssp585_p3_NLRes, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_ssp585_p3_NLRes")

## 计算diff245的拆解结果

In [49]:
diff245_pre_prime = IndR_ssp245_p3_pre_slope - IndR_his_pre_slope
term_diff245_1 = term_ssp245_p3_1 - term_his_1

# 测试4项合起来结果是否一致

u_diff245_bar = (u_ssp245_p3_bar - u_his_bar).sel(level=sel_level)
v_diff245_bar = (v_ssp245_p3_bar - v_his_bar).sel(level=sel_level)
wap_diff245_bar = (wap_ssp245_p3_bar - wap_his_bar).sel(level=sel_level)


IndR_diff245_uJJA_slope = IndR_ssp245_p3_uJJA_slope - IndR_his_uJJA_slope
IndR_diff245_vJJA_slope = IndR_ssp245_p3_vJJA_slope - IndR_his_vJJA_slope
IndR_diff245_wap_slope = IndR_ssp245_p3_wap_slope - IndR_his_wap_slope


hus_diff245_prime_dp = hus_ssp245_p3_prime_dp - hus_his_prime_dp
hus_diff245_prime_dx = hus_ssp245_p3_prime_dx - hus_his_prime_dx
hus_diff245_prime_dy = hus_ssp245_p3_prime_dy - hus_his_prime_dy

hus_diff245_bar_dp = hus_ssp245_p3_bar_dp - hus_his_bar_dp
hus_diff245_bar_dx = hus_ssp245_p3_bar_dx - hus_his_bar_dx
hus_diff245_bar_dy = hus_ssp245_p3_bar_dy - hus_his_bar_dy



### 水平热力项

In [33]:
# 水平热力项分解
#============================================================
#第一项将和his中的相减，合并为一项
term_diff245_c1_1_1 = vert_intergrate(u_his_bar.sel(level=sel_level)*hus_his_prime_dx.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_diff245_c1_1_2 = vert_intergrate(v_his_bar.sel(level=sel_level)*hus_his_prime_dy.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)

term_diff245_c1_1 = term_diff245_c1_1_1+term_diff245_c1_1_2
#============================================================
#第二项为固定nabla q prime为his情景，取v bar的情景变化
term_diff245_c1_2_1 = vert_intergrate(u_diff245_bar.sel(level=sel_level)*hus_his_prime_dx.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_diff245_c1_2_2 = vert_intergrate(v_diff245_bar.sel(level=sel_level)*hus_his_prime_dy.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)

term_diff245_c1_2 = term_diff245_c1_2_1+term_diff245_c1_2_2
#============================================================
#第三项为固定v bar为his情景，取nabla q prime的情景变化
term_diff245_c1_3_1 = vert_intergrate(u_his_bar.sel(level=sel_level)*hus_diff245_prime_dx.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_diff245_c1_3_2 = vert_intergrate(v_his_bar.sel(level=sel_level)*hus_diff245_prime_dy.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)

term_diff245_c1_3 = term_diff245_c1_3_1+term_diff245_c1_3_2
#============================================================
#第四项为v bar和nabla q prime都取情景变化
term_diff245_c1_4_1 = vert_intergrate(u_diff245_bar.sel(level=sel_level)*hus_diff245_prime_dx.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_diff245_c1_4_2 = vert_intergrate(v_diff245_bar.sel(level=sel_level)*hus_diff245_prime_dy.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)

term_diff245_c1_4 = term_diff245_c1_4_1+term_diff245_c1_4_2

In [41]:
ca.save(term_diff245_c1_1, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c1_1")
ca.save(term_diff245_c1_2, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c1_2")
ca.save(term_diff245_c1_3, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c1_3")
ca.save(term_diff245_c1_4, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c1_4")

### 水平动力项

In [42]:
# 水平动力项分解
#============================================================
#第一项为固定v prime，nabla q bar都为his的值，但在ssp245的气柱厚度下积分；需要与历史情境下的项合并
term_diff245_c2_1_1 = vert_intergrate(IndR_his_uJJA_slope.sel(level=sel_level)*hus_his_bar_dx.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_diff245_c2_1_2 = vert_intergrate(IndR_his_vJJA_slope.sel(level=sel_level)*hus_his_bar_dy.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_diff245_c2_1 = term_diff245_c2_1_1+term_diff245_c2_1_2

#============================================================
#第二项为固定nabla q bar为his情景，v prime取情景变化
term_diff245_c2_2_1 = vert_intergrate(IndR_diff245_uJJA_slope.sel(level=sel_level)*hus_his_bar_dx.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_diff245_c2_2_2 = vert_intergrate(IndR_diff245_vJJA_slope.sel(level=sel_level)*hus_his_bar_dy.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_diff245_c2_2 = term_diff245_c2_2_1+term_diff245_c2_2_2

#============================================================
#第三项为固定v prime为his情景，nabla q bar取情景变化
term_diff245_c2_3_1 = vert_intergrate(IndR_his_uJJA_slope.sel(level=sel_level)*hus_diff245_bar_dx.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_diff245_c2_3_2 = vert_intergrate(IndR_his_vJJA_slope.sel(level=sel_level)*hus_diff245_bar_dy.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_diff245_c2_3 = term_diff245_c2_3_1+term_diff245_c2_3_2

#============================================================
#第四项为v prime和nabla q bar均取情景变化
term_diff245_c2_4_1 = vert_intergrate(IndR_diff245_uJJA_slope.sel(level=sel_level)*hus_diff245_bar_dx.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_diff245_c2_4_2 = vert_intergrate(IndR_diff245_vJJA_slope.sel(level=sel_level)*hus_diff245_bar_dy.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)
term_diff245_c2_4 = term_diff245_c2_4_1+term_diff245_c2_4_2

In [43]:
ca.save(term_diff245_c2_1, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c2_1")
ca.save(term_diff245_c2_2, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c2_2")
ca.save(term_diff245_c2_3, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c2_3")
ca.save(term_diff245_c2_4, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c2_4")

### 垂直热力项

In [50]:
# 垂直热力项分解
#============================================================
#第一项为固定w bar，partial q prime partial p都为his的值，但在ssp245的气柱厚度下积分；需要与历史情境下的项合并
term_diff245_c3_1 = vert_intergrate(wap_his_bar.sel(level=sel_level)*hus_his_prime_dp.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)

#============================================================
#第二项为固定w bar为his情景，partial q prime partial p取情景变化
term_diff245_c3_2 = vert_intergrate(wap_his_bar.sel(level=sel_level)*hus_diff245_prime_dp.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)

#============================================================
#第三项为固定partial q prime partial p为his情景，w bar取情景变化
term_diff245_c3_3 = vert_intergrate(wap_diff245_bar.sel(level=sel_level)*hus_his_prime_dp.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)

#============================================================
#第四项为v prime和nabla q bar均取情景变化
term_diff245_c3_4 = vert_intergrate(wap_diff245_bar.sel(level=sel_level)*hus_diff245_prime_dp.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)


In [51]:
ca.save(term_diff245_c3_1, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c3_1")
ca.save(term_diff245_c3_2, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c3_2")
ca.save(term_diff245_c3_3, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c3_3")
ca.save(term_diff245_c3_4, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c3_4")

### 垂直动力项

In [47]:
# 垂直动力项分解
#============================================================
#第一项为固定w prime，partial q bar partial p都为his的值，但在ssp245的气柱厚度下积分；需要与历史情境下的项合并
term_diff245_c4_1 = vert_intergrate(IndR_his_wap_slope.sel(level=sel_level)*hus_his_bar_dp.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)

#============================================================
#第二项为固定w prime为his情景，partial q bar partial p取情景变化
term_diff245_c4_2 = vert_intergrate(IndR_his_wap_slope.sel(level=sel_level)*hus_diff245_bar_dp.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)

#============================================================
#第三项为固定partial q bar partial p为his情景，w prime取情景变化
term_diff245_c4_3 = vert_intergrate(IndR_diff245_wap_slope.sel(level=sel_level)*hus_his_bar_dp.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)

#============================================================
#第四项为w prime和partial q bar partial p均取情景变化
term_diff245_c4_4 = vert_intergrate(IndR_diff245_wap_slope.sel(level=sel_level)*hus_diff245_bar_dp.sel(level=sel_level).data, CMIP6dpg=ssp245_p3dpg)


In [48]:
ca.save(term_diff245_c4_1, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c4_1")
ca.save(term_diff245_c4_2, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c4_2")
ca.save(term_diff245_c4_3, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c4_3")
ca.save(term_diff245_c4_4, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_c4_4")

### 非线性与残差

In [52]:
term_diff245_NLRes = term_ssp245_p3_NLRes-term_his_NLRes
ca.save(term_diff245_NLRes, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff245_NLRes")

## 计算diff585的拆解结果

In [ ]:
diff585_pre_prime = IndR_ssp585_p3_pre_slope - IndR_his_pre_slope
term_diff585_1 = term_ssp585_p3_1 - term_his_1

# 测试4项合起来结果是否一致

u_diff585_bar = (u_ssp585_p3_bar - u_his_bar).sel(level=sel_level)
v_diff585_bar = (v_ssp585_p3_bar - v_his_bar).sel(level=sel_level)
wap_diff585_bar = (wap_ssp585_p3_bar - wap_his_bar).sel(level=sel_level)


IndR_diff585_uJJA_slope = IndR_ssp585_p3_uJJA_slope - IndR_his_uJJA_slope
IndR_diff585_vJJA_slope = IndR_ssp585_p3_vJJA_slope - IndR_his_vJJA_slope
IndR_diff585_wap_slope = IndR_ssp585_p3_wap_slope - IndR_his_wap_slope


hus_diff585_prime_dp = hus_ssp585_p3_prime_dp - hus_his_prime_dp
hus_diff585_prime_dx = hus_ssp585_p3_prime_dx - hus_his_prime_dx
hus_diff585_prime_dy = hus_ssp585_p3_prime_dy - hus_his_prime_dy

hus_diff585_bar_dp = hus_ssp585_p3_bar_dp - hus_his_bar_dp
hus_diff585_bar_dx = hus_ssp585_p3_bar_dx - hus_his_bar_dx
hus_diff585_bar_dy = hus_ssp585_p3_bar_dy - hus_his_bar_dy



### 水平热力项

In [ ]:
# 水平热力项分解
#============================================================
#第一项将和his中的相减，合并为一项
term_diff585_c1_1_1 = vert_intergrate(u_his_bar.sel(level=sel_level)*hus_his_prime_dx.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_diff585_c1_1_2 = vert_intergrate(v_his_bar.sel(level=sel_level)*hus_his_prime_dy.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)

term_diff585_c1_1 = term_diff585_c1_1_1+term_diff585_c1_1_2
#============================================================
#第二项为固定nabla q prime为his情景，取v bar的情景变化
term_diff585_c1_2_1 = vert_intergrate(u_diff585_bar.sel(level=sel_level)*hus_his_prime_dx.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_diff585_c1_2_2 = vert_intergrate(v_diff585_bar.sel(level=sel_level)*hus_his_prime_dy.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)

term_diff585_c1_2 = term_diff585_c1_2_1+term_diff585_c1_2_2
#============================================================
#第三项为固定v bar为his情景，取nabla q prime的情景变化
term_diff585_c1_3_1 = vert_intergrate(u_his_bar.sel(level=sel_level)*hus_diff585_prime_dx.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_diff585_c1_3_2 = vert_intergrate(v_his_bar.sel(level=sel_level)*hus_diff585_prime_dy.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)

term_diff585_c1_3 = term_diff585_c1_3_1+term_diff585_c1_3_2
#============================================================
#第四项为v bar和nabla q prime都取情景变化
term_diff585_c1_4_1 = vert_intergrate(u_diff585_bar.sel(level=sel_level)*hus_diff585_prime_dx.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_diff585_c1_4_2 = vert_intergrate(v_diff585_bar.sel(level=sel_level)*hus_diff585_prime_dy.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)

term_diff585_c1_4 = term_diff585_c1_4_1+term_diff585_c1_4_2

In [ ]:
ca.save(term_diff585_c1_1, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c1_1")
ca.save(term_diff585_c1_2, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c1_2")
ca.save(term_diff585_c1_3, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c1_3")
ca.save(term_diff585_c1_4, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c1_4")

### 水平动力项

In [ ]:
# 水平动力项分解
#============================================================
#第一项为固定v prime，nabla q bar都为his的值，但在ssp585的气柱厚度下积分；需要与历史情境下的项合并
term_diff585_c2_1_1 = vert_intergrate(IndR_his_uJJA_slope.sel(level=sel_level)*hus_his_bar_dx.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_diff585_c2_1_2 = vert_intergrate(IndR_his_vJJA_slope.sel(level=sel_level)*hus_his_bar_dy.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_diff585_c2_1 = term_diff585_c2_1_1+term_diff585_c2_1_2

#============================================================
#第二项为固定nabla q bar为his情景，v prime取情景变化
term_diff585_c2_2_1 = vert_intergrate(IndR_diff585_uJJA_slope.sel(level=sel_level)*hus_his_bar_dx.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_diff585_c2_2_2 = vert_intergrate(IndR_diff585_vJJA_slope.sel(level=sel_level)*hus_his_bar_dy.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_diff585_c2_2 = term_diff585_c2_2_1+term_diff585_c2_2_2

#============================================================
#第三项为固定v prime为his情景，nabla q bar取情景变化
term_diff585_c2_3_1 = vert_intergrate(IndR_his_uJJA_slope.sel(level=sel_level)*hus_diff585_bar_dx.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_diff585_c2_3_2 = vert_intergrate(IndR_his_vJJA_slope.sel(level=sel_level)*hus_diff585_bar_dy.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_diff585_c2_3 = term_diff585_c2_3_1+term_diff585_c2_3_2

#============================================================
#第四项为v prime和nabla q bar均取情景变化
term_diff585_c2_4_1 = vert_intergrate(IndR_diff585_uJJA_slope.sel(level=sel_level)*hus_diff585_bar_dx.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_diff585_c2_4_2 = vert_intergrate(IndR_diff585_vJJA_slope.sel(level=sel_level)*hus_diff585_bar_dy.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)
term_diff585_c2_4 = term_diff585_c2_4_1+term_diff585_c2_4_2

In [ ]:
ca.save(term_diff585_c2_1, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c2_1")
ca.save(term_diff585_c2_2, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c2_2")
ca.save(term_diff585_c2_3, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c2_3")
ca.save(term_diff585_c2_4, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c2_4")

### 垂直热力项

In [ ]:
# 垂直热力项分解
#============================================================
#第一项为固定w bar，partial q prime partial p都为his的值，但在ssp585的气柱厚度下积分；需要与历史情境下的项合并
term_diff585_c3_1 = vert_intergrate(wap_his_bar.sel(level=sel_level)*hus_his_prime_dp.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)

#============================================================
#第二项为固定w bar为his情景，partial q prime partial p取情景变化
term_diff585_c3_2 = vert_intergrate(wap_his_bar.sel(level=sel_level)*hus_diff585_prime_dp.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)

#============================================================
#第三项为固定partial q prime partial p为his情景，w bar取情景变化
term_diff585_c3_3 = vert_intergrate(wap_diff585_bar.sel(level=sel_level)*hus_his_prime_dp.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)

#============================================================
#第四项为v prime和nabla q bar均取情景变化
term_diff585_c3_4 = vert_intergrate(wap_diff585_bar.sel(level=sel_level)*hus_diff585_prime_dp.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)


In [ ]:
ca.save(term_diff585_c3_1, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c3_1")
ca.save(term_diff585_c3_2, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c3_2")
ca.save(term_diff585_c3_3, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c3_3")
ca.save(term_diff585_c3_4, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c3_4")

### 垂直动力项

In [ ]:
# 垂直动力项分解
#============================================================
#第一项为固定w prime，partial q bar partial p都为his的值，但在ssp585的气柱厚度下积分；需要与历史情境下的项合并
term_diff585_c4_1 = vert_intergrate(IndR_his_wap_slope.sel(level=sel_level)*hus_his_bar_dp.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)

#============================================================
#第二项为固定w prime为his情景，partial q bar partial p取情景变化
term_diff585_c4_2 = vert_intergrate(IndR_his_wap_slope.sel(level=sel_level)*hus_diff585_bar_dp.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)

#============================================================
#第三项为固定partial q bar partial p为his情景，w prime取情景变化
term_diff585_c4_3 = vert_intergrate(IndR_diff585_wap_slope.sel(level=sel_level)*hus_his_bar_dp.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)

#============================================================
#第四项为w prime和partial q bar partial p均取情景变化
term_diff585_c4_4 = vert_intergrate(IndR_diff585_wap_slope.sel(level=sel_level)*hus_diff585_bar_dp.sel(level=sel_level).data, CMIP6dpg=ssp585_p3dpg)


In [ ]:
ca.save(term_diff585_c4_1, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c4_1")
ca.save(term_diff585_c4_2, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c4_2")
ca.save(term_diff585_c4_3, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c4_3")
ca.save(term_diff585_c4_4, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_c4_4")

### 非线性与残差

In [ ]:
term_diff585_NLRes = term_ssp585_p3_NLRes-term_his_NLRes
ca.save(term_diff585_NLRes, "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/new_paper_data/qbudgetanalysis/term_diff585_NLRes")